# # 모듈 및 함수
---

In [2]:
import open3d as o3d
import numpy as np
import cv2
import torch
import pandas as pd
import re
import pickle as pkl
import json
import glob
import shutil
import os
from collections import Counter
from sklearn.model_selection import train_test_split
from scipy.spatial.transform import Rotation as R


# 문자열 숫자리스트로 바꾸는 함수
def str2list(txt):
    txt = txt.replace('\n', '').split(',')
    txt = list(map(float, txt))
    
    return txt


# 리스트를 문자열로 바꾸는 함수
def list2str(list):
    list = ' '.join(map(str, list))
    
    return list


# alpha 구하는 공식
import math

def normalizeAngle(angle):
    result = angle % (2*math.pi)
    if result < -math.pi:
        result += 2*math.pi
    elif result > math.pi:
        result -= 2*math.pi
    return result

def cal_alpha_ori(x, z, ry):  
    angle = ry
    angle -= -math.atan2(z, x) -1.5*math.pi 
    alpha = normalizeAngle(angle)
    return alpha # -1.818032754845337
# cal_alpha_ori(2.5702, 9.7190, -1.5595)


# convert 2d coordinate
def xyxy2xywhn(x, w=1920, h=1200, clip=False, eps=0.0):
    # Convert nx4 boxes from [x1, y1, x2, y2] to [x, y, w, h] normalized where xy1=top-left, xy2=bottom-right
    if clip:
        clip_boxes(x, (h - eps, w - eps))  # warning: inplace clip
    x = np.array(x).reshape(1, -1)
    y = np.copy(x)
    y[:, 0] = ((x[:, 0] + x[:, 2]) / 2) / w  # x center
    y[:, 1] = ((x[:, 1] + x[:, 3]) / 2) / h  # y center
    y[:, 2] = (x[:, 2] - x[:, 0]) / w  # width
    y[:, 3] = (x[:, 3] - x[:, 1]) / h  # height
    y = list(y.reshape(-1))
    return y

# rotation matrix
def roty(t, Rx=90/180*np.pi):
    ''' Rotation about the y-axis. '''
    c = np.cos(t)
    s = np.sin(t)
    # return  np.array([[c, 0, s],
    #                 [0, 1, 0],
    #                 [-s, 0, c]])

    X = np.array([[1, 0, 0],
                    [0, np.cos(Rx), -np.sin(Rx)],
                    [0, np.sin(Rx), np.cos(Rx)]])

    # X = np.eye(3)

    Z = np.array([[c, -s, 0],
                    [s, c, 0],
                    [0, 0, 1]])
    
    return np.matmul(Z, X)

# # kitti dataset calib 확인
---

In [ ]:
with open('/data/hwang/datasets/kitti/training/calib/000000.txt', 'r') as f:
    txt = f.readlines()

p0 = txt[0].replace('\n', '').split(' ')[1:] # 0번 카메라, projection matrix (= intrinsic)
p1 = txt[1].replace('\n', '').split(' ')[1:] # 1번 카메라
p2 = txt[2].replace('\n', '').split(' ')[1:] # 2번 카메라 (모델에 이 정보를 사용), (i, 4)위치는 기본 카메라와의 위상 차이 (각 x, y, z 축)
p3 = txt[3].replace('\n', '').split(' ')[1:] # 3번 카메라
R_rect = txt[4].replace('\n', '').split(' ')[1:] # 스테레오 카메라로 촬영 시 rotation matrix 보정 수치 (모노 카메라면 단위행렬 사용)
Tr_velo = txt[5].replace('\n', '').split(' ')[1:] # extrinsic (사용은 rigid body transformation 형태로 사용, extrinsic 4행에 [0, 0, 0, 1]을 추가)
Tr_imu = txt[6].replace('\n', '').split(' ')[1:]

print(np.around(np.asarray(p0, dtype=float).reshape(-1, 4), 1), '\n')
print(np.around(np.asarray(p1, dtype=float).reshape(-1, 4), 1), '\n')
print(np.around(np.asarray(p2, dtype=float).reshape(-1, 4), 1), '\n')
print(np.around(np.asarray(p3, dtype=float).reshape(-1, 4), 1), '\n')
print(np.asarray(R_rect, dtype=float).reshape(-1, 3), '\n')
print(np.asarray(Tr_velo, dtype=float).reshape(-1, 4), '\n')
print(np.asarray(Tr_imu, dtype=float).reshape(-1, 4), '\n')

In [ ]:
calib_json = {
    "extrinsic": np.asarray(Tr_velo, dtype=np.float32).tolist(),
    "intrinsic": np.asarray(p2, dtype=np.float32).reshape(-1, 4)[:3, :3].flatten().tolist()
}
json_object = json.dumps(calib_json, indent = 4) 
# Writing to sample.json 
with open('/data/NIA50/SUSTechPOINTS_2-050/data/nia48/calib/000055.json', "w") as outfile: 
    outfile.write(json_object) 

In [ ]:
calib_path = '/data/hwang/datasets/kitti/training/calib/'
frame = '000000'

def get_inv_matrix(file, v2c, rect):
    with open(file) as f:
        lines = f.readlines()
        trans = [x for x in filter(lambda s: s.startswith(v2c), lines)][0]
        
        matrix = [m for m in map(lambda x: float(x), trans.strip().split(" ")[1:])]
        matrix = matrix + [0,0,0,1]
        m = np.array(matrix)
        velo_to_cam  = m.reshape([4,4])


        trans = [x for x in filter(lambda s: s.startswith(rect), lines)][0]
        matrix = [m for m in map(lambda x: float(x), trans.strip().split(" ")[1:])]        
        m = np.array(matrix).reshape(3,3)
        
        m = np.concatenate((m, np.expand_dims(np.zeros(3), 1)), axis=1)
        
        rect = np.concatenate((m, np.expand_dims(np.array([0,0,0,1]), 0)), axis=0)        
        
        # print(velo_to_cam, rect)    
        m = np.matmul(rect, velo_to_cam)


        m = np.linalg.inv(m)
        
        return m
        
def get_detection_inv_matrix(calib_path, frame):
    file = os.path.join(calib_path, frame+".txt")
    return get_inv_matrix(file, "Tr_velo_to_cam", "R0_rect")

# # convert pcd - npy
---

## # pcd to npy
---

In [ ]:
# device = o3d.core.Device('cuda:1')
pcd_f = o3d.t.io.read_point_cloud('/data/NIA50/SUSTechPOINTS_2-050/data/Suwon_A_0000/lidar/0000.pcd')
print(pcd_f)

positions = pcd_f.point.positions.numpy()
intensity = pcd_f.point.intensity.numpy()
pcd = np.concatenate((positions, intensity), axis = 1)

pcd

# np.frombuffer(pcd.tobytes(), dtype='float32').reshape(-1, 4) # bytes에서 numpy로 복원

## # npy to pcd
---

In [ ]:
pcd_f = o3d.t.io.read_point_cloud('/data/NIA50/SUSTechPOINTS_2-050/data/Suwon_A_0000/lidar/0000.pcd')

positions = pcd_f.point.positions.numpy()
intensity = pcd_f.point.intensity.numpy()
pcd = np.concatenate((positions, intensity), axis = 1)

xyzi = np.stack([pcd[:,1]*-1, pcd[:, 0], pcd[:, 2], pcd[:, 3]], axis=1)

xyz = xyzi[:, :3]
i = [[i] for i in xyzi[:, 3]]

pcd = o3d.t.geometry.PointCloud()
pcd.point["positions"] = o3d.core.Tensor(xyz)
pcd.point["intensity"] = o3d.core.Tensor(i)

o3d.t.io.write_point_cloud('/data/NIA50/SUSTechPOINTS_2-050/data/Suwon_A_0000/lidar/0011.pcd', pcd, write_ascii=True)

In [ ]:
xyzi = np.fromfile('/data/hwang/datasets/kitti/training/velodyne/000000.bin', dtype=np.float32).reshape(-1, 4)

xyz = xyzi[:, :3]
i = [[i] for i in xyzi[:, 3]]

pcd = o3d.t.geometry.PointCloud()
pcd.point["positions"] = o3d.core.Tensor(xyz)
pcd.point["intensity"] = o3d.core.Tensor(i)

o3d.t.io.write_point_cloud('/data/NIA50/SUSTechPOINTS_2-050/data/Suwon_A_0000/lidar/0010.pcd', pcd)

# # Fundamental Matrix
---

In [ ]:
# !pip install opencv-contrib-python # cv2.xfeatures2d.sift_create()를 사용하기 위해서는 따로 설치 필요.

In [ ]:
# 2개의 이미지 사이에 매칭되는 특징점을 최대한 많이 찾아내야 한다.
# FLANN에 기반한 매처를 이용한 SIFT 디스크립터를 사용하고 ratio를 테스트한다.

import cv2
import numpy as np
import matplotlib.pyplot as plt

img1 = cv2.imread('/data/NIA50/50-2/data/NIA48/fine_data/S_Clip_02244_02/Camera/CameraFront/blur/2-048_02244_CF_001.png')
img2 = cv2.imread('/data/NIA50/50-2/data/NIA48/fine_data/S_Clip_02244_02/Camera/CameraFront/blur/2-048_02244_CF_032.png')

sift = cv2.SIFT_create()

# find the keypoints and descriptors with SIFT
kp1, des1 = sift.detectAndCompute(img1, None)
kp2, des2 = sift.detectAndCompute(img2, None)

# FLANN parameters
FLANN_INDEX_KDTREE = 0
index_params = {'algorithm': FLANN_INDEX_KDTREE, 'trees': 5}
search_params = {'check': 50}

flann = cv2.FlannBasedMatcher(index_params, search_params)
matches = flann.knnMatch(des1, des2, k=2)

good = []
pts1 = []
pts2 = []

# ratio test as per Lowe's paper
for i, (m, n) in enumerate(matches):
    if m.distance < 0.8*n.distance:
        good.append(m)
        pts2.append(kp2[m.trainIdx].pt)
        pts1.append(kp1[m.queryIdx].pt)

In [ ]:
# 두 이미지로부터 매칭되는 특징점 중 가장 좋은 것들을 이용해 Fundamental Matrix를 계산한다.

pts1 = np.int32(pts1)
pts2 = np.int32(pts2)
F, mask = cv2.findFundamentalMat(pts1, pts2, cv2.FM_LMEDS)

# We select only inlier points
pts1 = pts1[mask.ravel()==1]
pts2 = pts2[mask.ravel()==1]

# Fundamental Matrix
print(F)

In [ ]:
# Epiline 찾기

def drawlines(img1, img2, lines, pts1, pts2):
    '''img1 - image on which we draw epilines for the points in img2 \n
    lines - corresponding epilines'''    
    
    r, c, _ = img1.shape
    # img1 = cv2.cvtColor(img1, cv2.COLOR_GRAY2BGR)
    # img2 = cv2.cvtColor(img2, cv2.COLOR_GRAY2BGR)
    
    for r, pt1, pt2 in zip(lines, pts1, pts2):
        color = tuple(np.random.randint(0,255,3).tolist())
        x0,y0 = map(int, [0, -r[2]/r[1] ])
        x1,y1 = map(int, [c, -(r[2]+r[0]*c)/r[1] ])
        img1 = cv2.line(img1, (x0,y0), (x1,y1), color, 1)
        img1 = cv2.circle(img1, tuple(pt1), 5, color, -1)
        img2 = cv2.circle(img2, tuple(pt2), 5, color, -1)
        
    return img1, img2

In [ ]:
# Epiline을 찾아 그려준다.

# Find epilines corresponding to points in right image (second image) and
# drawing its lines on left image
lines1 = cv2.computeCorrespondEpilines(pts2.reshape(-1,1,2), 2,F)
lines1 = lines1.reshape(-1,3)
img5,img6 = drawlines(img1,img2,lines1,pts1,pts2)
# Find epilines corresponding to points in left image (first image) and
# drawing its lines on right image
lines2 = cv2.computeCorrespondEpilines(pts1.reshape(-1,1,2), 1,F)
lines2 = lines2.reshape(-1,3)
img3,img4 = drawlines(img2, img1, lines2, pts2, pts1)

plt.figure(figsize=(20, 15))
plt.subplot(121),plt.imshow(img5)
plt.subplot(122),plt.imshow(img3)
plt.show()

In [ ]:
np.around(F, 3)xx

# # Essential Matrix
---

In [ ]:
E, mask = cv2.findEssentialMat(points1=pts1, 
                               points2=pts2, 
                               cameraMatrix=intrinsic[:3, :3],
                               method = cv2.RANSAC,
                               prob = 0.999,
                               threshold = 1.0)

E

In [ ]:
np.around(np.matmul(E, F), 4)

In [ ]:
_, r, t, _ = cv2.recoverPose(E, pts1, pts2, intrinsic[:3, :3])

In [ ]:
print(np.around(E, 4))

In [ ]:
F

In [ ]:
E

In [ ]:
rotation_matrix

# # 특수환경 자율주행 3D 이미지
---

## # make dataframe
---

In [ ]:
# 검색용 코드
df.loc[df['xyzlwh'].apply(lambda x: -31.16 in x[:3])]

### # training
---

In [ ]:
train_df = pd.DataFrame(columns=['clip', 'frame', 'xyxy', 'xyzlwh', 'theta', 'eulerangle', 'translation', 'fxfycxcy', 'k1k2p1p2', 'pcd', 'class'])
idx = 0

path = '/data/NIA50/data/특수환경 자율주행 3D 이미지/training/'
clips = sorted([i for i in os.listdir(path) if 'drive_' in i])
# clip = clips[20]

for clip in clips:
    try:
        with open(path+clip+'/calib.txt', 'r') as f:
            calib = [re.sub('\n', '', i) for i in f.readlines()]

        eulerangle = calib[4].split(',')
        translation = calib[6].split(',')
        intrinsic = calib[8].split(',')
        fxfycxcy = intrinsic[:4]
        k1k2p1p2 = intrinsic[4:]

        metas = sorted(os.listdir(path+clip+'/meta/'))

        for meta in metas:
            meta = path+clip+f'/meta/{meta}'
            frame = meta[-11:-5]

            pcd_f = o3d.t.io.read_point_cloud(path+clip+f'/lidar/{clip}_{frame}.pcd')
            positions = pcd_f.point.positions.numpy()
            intensity = pcd_f.point.intensity.numpy()
            pcd = np.concatenate((positions, intensity), axis = 1)

            with open(meta, 'r') as f:
                meta_js = json.load(f)

            objects = meta_js['OBJECT_LIST'][0]['3D_LIST']
            for i in range(len(objects)):
                object = objects[i]
                xyxy = object['BOX']
                xyz = object['LOCATION']
                hwl = object['DIMENSION']
                xyzlwh = xyz + hwl[2:3] + hwl[1:2] + hwl[0:1]
                # xyzlwh = ', '.join(map(str, xyzlwh))
                theta = object['YAW']
                class_ = object['TYPE']

                train_df.loc[idx] = [clip, frame, xyxy, xyzlwh, theta, eulerangle, translation, fxfycxcy, k1k2p1p2, pcd, class_]

                idx+=1
    except:
        pass
    
    train_df.to_csv('/data/NIA50/data/특수환경 자율주행 3D 이미지/train_df.csv', index=False)

### # validation
---

In [ ]:
val_df = pd.DataFrame(columns=['clip', 'frame', 'xyxy', 'xyzlwh', 'theta', 'eulerangle', 'translation', 'fxfycxcy', 'k1k2p1p2', 'class', 'type'])
idx = 0

path = '/data/NIA50/data/특수환경 자율주행 3D 이미지/Validation/'
clips = sorted([i for i in os.listdir(path) if 'drive_' in i])
# clip = clips[20]

for clip in clips:
    try:
        with open(path+clip+'/calib.txt', 'r') as f:
            calib = [re.sub('\n', '', i) for i in f.readlines()]

        eulerangle = list(map(float, calib[4].split(',')))
        translation = list(map(float, calib[6].split(',')))
        intrinsic = list(map(float, calib[8].split(',')))
        fxfycxcy = intrinsic[:4]
        k1k2p1p2 = intrinsic[4:]

        metas = sorted(os.listdir(path+clip+'/lidar/'))

        for meta in metas:
            meta = path+clip+f'/meta/{meta}'
            frame = meta[-11:-5]

            # pcd_f = o3d.t.io.read_point_cloud(path+clip+f'/lidar/{clip}_{frame}.pcd')
            # positions = pcd_f.point.positions.numpy()
            # intensity = pcd_f.point.intensity.numpy()
            # pcd = np.concatenate((positions, intensity), axis = 1).tobytes()

            with open(meta, 'r') as f:
                meta_js = json.load(f)

            objects = meta_js['OBJECT_LIST'][0]['3D_LIST']
            for i in range(len(objects)):
                object = objects[i]
                xyxy = object['BOX']
                xyz = object['LOCATION']
                hwl = object['DIMENSION']
                xyzlwh = xyz + hwl[2:3] + hwl[1:2] + hwl[0:1]
                # xyzlwh = ', '.join(map(str, xyzlwh))
                theta = object['YAW']
                class_ = object['CLASS']
                type_ = object['TYPE']

                val_df.loc[idx] = [clip, frame, xyxy, xyzlwh, theta, eulerangle, translation, fxfycxcy, k1k2p1p2, class_, type_]

                idx+=1
    except:
        pass
    
    val_df.to_csv('/data/NIA50/data/특수환경 자율주행 3D 이미지/val_df.csv', index=False)

## # PV-RCNN (fit custom dataset)
---

### # save npy
---

In [ ]:
val_df = pd.read_csv('/data/NIA50/data/특수환경 자율주행 3D 이미지/val_df.csv')
val_df['frame'] = val_df['frame'].apply(lambda x: str(x).zfill(6))
val_df.head()

In [ ]:
path = '/data/NIA50/data/특수환경 자율주행 3D 이미지/Validation/'
save_path = '/data/NIA50/data/특수환경 자율주행 3D 이미지/mm_Train3D/points'

true_data = val_df.loc[val_df['xyzlwh']!='[0.0, 0.0, 0.0, 0, 0, 0]']
clips = true_data['clip'].unique()

for clip in clips:
    frames = true_data.loc[true_data['clip'] == clip, 'frame'].unique()

    for frame in frames:
        try:
            pcd_name = pcd_name = clip + f'_{frame}.pcd'
            pcd_f = o3d.t.io.read_point_cloud(path+clip+'/lidar/'+pcd_name)

            positions = pcd_f.point.positions.numpy()
            intensity = pcd_f.point.intensity.numpy()
            pcd = np.concatenate((positions, intensity), axis = 1)

            npy = np.save(save_path+f'/{clip}_{frame}.npy', pcd)
        except:
            pass

In [ ]:
path = '/data/NIA50/data/특수환경 자율주행 3D 이미지/mm_Train3D/ImageSets'

### # dividing train, val
---

#### # imagesets
---

In [ ]:
val_df = pd.read_csv('/data/NIA50/data/특수환경 자율주행 3D 이미지/val_df.csv')
val_df['frame'] = val_df['frame'].apply(lambda x: str(x).zfill(6))
tdf = val_df.loc[val_df['xyzlwh']!='[0.0, 0.0, 0.0, 0, 0, 0]']
tdf.head()

In [ ]:
images = []

clips = tdf['clip'].unique()
for clip in clips:

    frames = tdf.loc[tdf['clip'] == clip, 'frame'].unique()
    for frame in frames:
        image = clip+f'_{frame}'
        images.append(image)

len(images), len(os.listdir('/data/NIA50/data/특수환경 자율주행 3D 이미지/mm_Train3D/points'))

In [ ]:
images = sorted([re.sub('.npy', '', i) for i in os.listdir('/data/NIA50/data/특수환경 자율주행 3D 이미지/mm_Train3D/points')])
len(images)

In [ ]:
path = '/data/NIA50/data/특수환경 자율주행 3D 이미지/mm_Train3D/ImageSets/'

train, val = train_test_split(images, test_size=0.2, random_state=0)

with open(path+'train.txt', 'w') as f:
    f.write('\n'.join(sorted(train)))

with open(path+'val.txt', 'w') as f:
    f.write('\n'.join(sorted(val)))

#### # labels
---

In [ ]:
# 차집합

set(labels) - set(images)
set(labels).difference(set(images))

In [ ]:
path = '/data/NIA50/data/특수환경 자율주행 3D 이미지/mm_Train3D/labels/'
labels = [re.sub('.txt', '', i) for i in os.listdir(path)]

for n_pcd in set(labels) - set(images):
    if os.path.exists(path+f'{n_pcd}.txt'):
        os.remove(path+f'{n_pcd}.txt')

#### # calibration
---

In [ ]:
from scipy.spatial.transform import Rotation as R


path = '/data/NIA50/data/특수환경 자율주행 3D 이미지/mm_Train3D/calib/'

clips = tdf['clip'].unique()
for clip in clips:

    frames = tdf.loc[tdf['clip'] == clip, 'frame'].unique()
    for frame in frames:
        image = clip+f'_{frame}'

        fxfycxcy = json.loads(tdf.loc[(tdf['clip']==clip) & (tdf['frame']==frame), 'fxfycxcy'].values[0])
        eulerangle = json.loads(tdf.loc[(tdf['clip']==clip) & (tdf['frame']==frame), 'eulerangle'].values[0])
        translation = json.loads(tdf.loc[(tdf['clip']==clip) & (tdf['frame']==frame), 'translation'].values[0])
        intrinsic = np.asarray([[fxfycxcy[0], 0, fxfycxcy[2]], [0, fxfycxcy[1], fxfycxcy[3]], [0, 0, 1]])
        rotation_matrix = R.from_euler('xyz', eulerangle, degrees=True).as_matrix()

        p0 = np.hstack([intrinsic, np.zeros((3, 1))])
        R0_rect = np.eye(3)
        Tr_velo_to_cam = np.hstack([rotation_matrix, np.asarray(translation).reshape(3, -1)])
        Tr_imu_to_velo = np.zeros((12))

        # to string
        p0 = ' '.join(map(str, p0.reshape(-1).tolist()))
        p1 = p0
        p2 = p0
        p3 = p0
        R0_rect = ' '.join(map(str, R0_rect.reshape(-1).tolist()))
        Tr_velo_to_cam = ' '.join(map(str, Tr_velo_to_cam.reshape(-1).tolist()))
        Tr_imu_to_velo = ' '.join(map(str, Tr_imu_to_velo.reshape(-1).tolist()))

        label_lines = ['P0: '+p0, 'P1: '+p1, 'P2: '+p2, 'P3: '+p3, 'R0_rect: '+R0_rect, 'Tr_velo_to_cam: '+Tr_velo_to_cam, 'Tr_imu_to_velo: '+Tr_imu_to_velo]
        with open(path + f'{image}.txt', 'w') as f:
            f.write('\n'.join(label_lines))

            # f.write(f'P0: {p0}\n')
            # f.write(f'P1: {p1}\n')
            # f.write(f'P2: {p2}\n')
            # f.write(f'P3: {p3}\n')
            # f.write(f'R0_rect: {R0_rect}\n')
            # f.write(f'Tr_velo_to_cam: {Tr_velo_to_cam}\n')
            # f.write(f'Tr_imu_to_velo: {Tr_imu_to_velo}')

In [ ]:
path = '/data/NIA50/data/특수환경 자율주행 3D 이미지/mm_Train3D/calib/'
labels = [re.sub('.txt', '', i) for i in os.listdir(path)]

for n_pcd in set(labels) - set(images):
    if os.path.exists(path+f'{n_pcd}.txt'):
        os.remove(path+f'{n_pcd}.txt')

## # PointRCNN (fit kitti datset)
---

In [ ]:
val_df = pd.read_csv('/data/NIA50/data/특수환경 자율주행 3D 이미지/val_df.csv')
val_df['frame'] = val_df['frame'].apply(lambda x: str(x).zfill(6))
images = []
for i in val_df.index.values:
    image = val_df.loc[i]['clip'] + '_' + val_df.loc[i]['frame']
    images.append(image)
val_df['image'] = images
tdf = val_df.loc[val_df['xyzlwh']!='[0.0, 0.0, 0.0, 0, 0, 0]']
tdf.head(3)

### # imagesets
---

In [ ]:
# 라벨 복사

import glob
import shutil

exi_pcd = glob.glob('/data/NIA50/data/특수환경 자율주행 3D 이미지/Validation/*/lidar/*.pcd')
exi_pcd = [i[-21:-4] for i in exi_pcd]

images_true = []
for clip in tdf['clip'].unique():
    for frame in tdf.loc[tdf['clip'] == clip, 'frame'].unique():
        image = clip + f'_{frame}'
        images_true.append(image)
        
images = sorted(set(images_true) & set(exi_pcd))
# images = sorted([re.sub('.npy', '', i) for i in os.listdir('/data/NIA50/data/특수환경 자율주행 3D 이미지/mm_Train3D/points')])

images_re, test = train_test_split(images, test_size = 0.1, random_state = 0)
train, val = train_test_split(images_re, test_size = 0.2, random_state = 0)

path = '/data/NIA50/data/특수환경 자율주행 3D 이미지/pointrcnn_Train3D/imagesets/'

# with open(path+'train.txt', 'w') as f:
#     f.write('\n'.join(sorted(train)))

# with open(path+'val.txt', 'w') as f:
#     f.write('\n'.join(sorted(val)))
    
# with open(path+'test.txt', 'w') as f:
#     f.write('\n'.join(sorted(test)))

In [ ]:
len(train), len(val), len(test)

### # velodyne
---

In [ ]:
pcd_files = glob.glob('/data/NIA50/data/특수환경 자율주행 3D 이미지/Validation/*/lidar/*.pcd')

pcd_path = []
for pcd_file in pcd_files:
    for image in images:
        if image == pcd_file[-21:-4]:
            pcd_path.append(pcd_file)

In [ ]:
os.makedirs('/data/NIA50/data/특수환경 자율주행 3D 이미지/pointrcnn_Train3D/training/velodyne', exist_ok=True)
os.makedirs('/data/NIA50/data/특수환경 자율주행 3D 이미지/pointrcnn_Train3D/testing/velodyne', exist_ok=True)

for pcd_file in pcd_path:
    pcd_f = o3d.t.io.read_point_cloud(pcd_file)
    positions = pcd_f.point.positions.numpy()
    intensity = pcd_f.point.intensity.numpy()
    
    pcd = np.concatenate((positions, intensity), axis = 1)
    pcd_bytes = pcd.tobytes()
    
    if pcd_file[-21:-4] in train:
        with open(f'/data/NIA50/data/특수환경 자율주행 3D 이미지/pointrcnn_Train3D/training/velodyne/{pcd_file[-21:-4]}.bin', 'wb') as f:
            f.write(pcd_bytes)
            
    elif pcd_file[-21:-4] in val:
        with open(f'/data/NIA50/data/특수환경 자율주행 3D 이미지/pointrcnn_Train3D/training/velodyne/{pcd_file[-21:-4]}.bin', 'wb') as f:
            f.write(pcd_bytes)

    elif pcd_file[-21:-4] in test:
        with open(f'/data/NIA50/data/특수환경 자율주행 3D 이미지/pointrcnn_Train3D/testing/velodyne/{pcd_file[-21:-4]}.bin', 'wb') as f:
            f.write(pcd_bytes)

### # calib
---

In [ ]:
np.asarray([[0, 1, 0],
            [0, 0, 1],
            [1, 0, 0]])

In [ ]:
from scipy.spatial.transform import Rotation as R


clips = tdf['clip'].unique()
for clip in clips[:1]:

    frames = tdf.loc[tdf['clip'] == clip, 'frame'].unique()
    for frame in frames:
        image = clip+f'_{frame}'

        fxfycxcy = json.loads(tdf.loc[(tdf['clip']==clip) & (tdf['frame']==frame), 'fxfycxcy'].values[0])
        # eulerangle = json.loads(tdf.loc[(tdf['clip']==clip) & (tdf['frame']==frame), 'eulerangle'].values[0]) * np.asarray(180/np.pi)
        eulerangle = json.loads(tdf.loc[(tdf['clip']==clip) & (tdf['frame']==frame), 'eulerangle'].values[0])
        translation = json.loads(tdf.loc[(tdf['clip']==clip) & (tdf['frame']==frame), 'translation'].values[0])
        intrinsic = np.asarray([[fxfycxcy[0], 0, fxfycxcy[2]], [0, fxfycxcy[1], fxfycxcy[3]], [0, 0, 1]])
        rotation_matrix = R.from_euler('xyz', eulerangle, degrees=True).as_matrix()

        p0 = np.hstack([intrinsic, np.zeros((3, 1))])
        R0_rect = np.eye(3)
        # velo_to_cam_rotation = np.asarray([[0, 1, 0],
        #                                    [0, 0, 1],
        #                                    [1, 0, 0]])
        Tr_velo_to_cam = np.hstack([rotation_matrix, np.asarray(translation).reshape(3, -1)])
        Tr_imu_to_velo = np.zeros((12))

        # to string
        p0 = ' '.join(map(str, p0.reshape(-1).tolist()))
        p1 = p0
        p2 = p0
        p3 = p0
        R0_rect = ' '.join(map(str, R0_rect.reshape(-1).tolist()))
        # R0_rect = ' '.join(map(str, rotation_matrix.reshape(-1).tolist()))
        Tr_velo_to_cam = ' '.join(map(str, Tr_velo_to_cam.reshape(-1).tolist()))
        Tr_imu_to_velo = ' '.join(map(str, Tr_imu_to_velo.reshape(-1).tolist()))

        label_lines = ['P0: '+p0, 'P1: '+p1, 'P2: '+p2, 'P3: '+p3, 'R0_rect: '+R0_rect, 'Tr_velo_to_cam: '+Tr_velo_to_cam, 'Tr_imu_to_velo: '+Tr_imu_to_velo]
                
        # if image in train:
        #     with open('/data/NIA50/data/특수환경 자율주행 3D 이미지/pointrcnn_Train3D/training/calib/' + f'{image}.txt', 'w') as f:
        #         f.write('\n'.join(label_lines))
                
        if image in sorted(val)[:1]:
            with open('/data/NIA50/data/특수환경 자율주행 3D 이미지/pointrcnn_Train3D/training/calib/' + f'{image}.txt', 'w') as f:
                f.write('\n'.join(label_lines))
                
        # elif image in test:
        #     with open('/data/NIA50/data/특수환경 자율주행 3D 이미지/pointrcnn_Train3D/testing/calib/' + f'{image}.txt', 'w') as f:
        #         f.write('\n'.join(label_lines))
                
            # f.write(f'P0: {p0}\n')
            # f.write(f'P1: {p1}\n')
            # f.write(f'P2: {p2}\n')
            # f.write(f'P3: {p3}\n')
            # f.write(f'R0_rect: {R0_rect}\n')
            # f.write(f'Tr_velo_to_cam: {Tr_velo_to_cam}\n')
            # f.write(f'Tr_imu_to_velo: {Tr_imu_to_velo}')

### # label_2
---

In [ ]:
# alpha 구하는 공식

import math

def normalizeAngle(angle):
    result = angle % (2*math.pi)
    if result < -math.pi:
        result += 2*math.pi
    elif result > math.pi:
        result -= 2*math.pi
    return result

def cal_alpha_ori(x, z, ry):  
    angle = ry
    angle -= -math.atan2(z, x) -1.5*math.pi 
    alpha = normalizeAngle(angle)
    return alpha # -1.818032754845337

cal_alpha_ori(2.5702, 9.7190, -1.5595)

In [ ]:
os.makedirs('/data/NIA50/data/특수환경 자율주행 3D 이미지/pointrcnn_Train3D/training/label_2', exist_ok=True)

idx = tdf.index.values

for i in idx[:1]:
    clip = tdf.loc[i]['clip']
    frame = tdf.loc[i]['frame']
    image = clip + f'_{frame}'
    
    type_ = tdf.loc[i]['class']
    truncation = 0.00
    occulsion = 0
    xyxy = json.loads(tdf.loc[i]['xyxy'])
    xyzlwh = json.loads(tdf.loc[i]['xyzlwh'])
    hwlxyz = [xyzlwh[5], xyzlwh[4], xyzlwh[3], xyzlwh[0], xyzlwh[1], xyzlwh[2]]
    rotation_y = tdf.loc[i]['theta'] # rotation_y = theta
    alpha = np.around(cal_alpha_ori(xyzlwh[0]*-1, xyzlwh[2], rotation_y), 2)
    
    if type_ == 'VEHICLE': type_ = 'Car'
    elif type_ == 'BICYCLE': type_ = 'Cyclist'
    elif type_ == 'PEDESTRIAN': type_ = 'Pedestrian'
    
    if xyxy[0] < 0: xyxy[0] = 0
    if xyxy[1] < 0: xyxy[1] = 0
    if xyxy[2] > 1920: xyxy[2] = 1920
    if xyxy[3] > 1200: xyxy[3] = 1200
    
    xyxy = ' '.join(map(str, xyxy))
    hwlxyz = ' '.join(map(str, hwlxyz))
    
    label = list(map(str, [type_, truncation, occulsion, alpha, xyxy, hwlxyz, rotation_y]))
    
    if image in train + val:
        with open(f'/data/NIA50/data/특수환경 자율주행 3D 이미지/pointrcnn_Train3D/training/label_2/{image}.txt', 'w') as f:
            f.write(' '.join(label))
            f.write('\n')

In [ ]:
# clips = tdf['clip'].unique()
# for clip in clips[:1]:
    
#     frames = tdf.loc[tdf['clip']==clip, 'frame'].unique()
#     for frame in frames:
        
#         image = clip+f'_{frame}'
#         for idx in len(tdf.loc[(tdf['clip']==clip) & (tdf['frame']==frame))
#         class_ = tdf.loc[(tdf['clip']==clip) & (tdf['frame']==frame), 'class']

### # image_2
---

In [ ]:
os.makedirs('/data/NIA50/data/특수환경 자율주행 3D 이미지/pointrcnn_Train3D/training/image_2', exist_ok=True)
os.makedirs('/data/NIA50/data/특수환경 자율주행 3D 이미지/pointrcnn_Train3D/testing/image_2', exist_ok=True)

for train_img in train + val:
    copy_path = glob.glob(f'/data/NIA50/data/특수환경 자율주행 3D 이미지/Validation/*/image_0/{train_img}.jpg')[0]
    paste_path = f'/data/NIA50/data/특수환경 자율주행 3D 이미지/pointrcnn_Train3D/training/image_2/{train_img}.jpg'
    shutil.copy(copy_path, paste_path)
    
# for test_img in test:
#     copy_path = glob.glob(f'/data/NIA50/data/특수환경 자율주행 3D 이미지/Validation/*/image_0/{test_img}.jpg')[0]
#     paste_path = f'/data/NIA50/data/특수환경 자율주행 3D 이미지/pointrcnn_Train3D/testing/image_2/{test_img}.jpg'
#     shutil.copy(copy_path, paste_path)

# # NIA48
---
- PointRCNN

In [ ]:
def get_cropped_and_resized_intrinsic(
    fx, fy, cx, cy, crop_cx, crop_cy, resize_fx, resize_fy):
    '''
    crop_cx : crop size of u axis orientation in image plane
    crop_cy : crop size of v axis orientation in image plane
    resize_fx : resize ratio of width orientation in image plane
    resize_fy : resize ratio of height orientation in image plane    
    '''

    cx -= crop_cx
    cy -= crop_cy

    fx *= resize_fx
    fy *= resize_fy

    cx *= resize_fx
    cy *= resize_fy

    return fx, fy, cx, cy

In [ ]:
# 문자열 숫자리스트로 바꾸는 함수
def str_cng(txt):
    txt = txt.replace('\n', '').split(',')
    txt = list(map(float, txt))
    
    return txt

## # calib
---

In [ ]:

path = '/data/NIA50/50-2/data/NIA48/training/calib/'
os.makedirs(path, exist_ok=True)

with open('/data/NIA50/50-2/data/NIA48/fine_data/S_Clip_02244_02/calib/Lidar_camera_calib/2-048_02244_LCC_CF.txt') as f:
    camera_calib = f.readlines()
# with open('/data/NIA48/S_Clip_03400_06/calib/Lidar_radar_calib/2-048_03400_LRC_RF.txt') as f:
#     radar_calib = f.readlines()

essential_matrix = np.asarray([[0, 1, 0],
                               [0, 0, 1],
                               [1, 0, 0]])
# essential_matrix = E
eulerangle = str2list(camera_calib[4]) # z, x, y
# eulerangle = np.asarray(eulerangle) + np.asarray([-360*np.pi/180, -90*np.pi/180, -90*np.pi/180])
X = R.from_euler('X', eulerangle[1]).as_matrix()
Y = R.from_euler('Y', eulerangle[0]).as_matrix()
Z = R.from_euler('Z', eulerangle[2]).as_matrix()
rotation_matrix = Z @ Y @ X
# rotation_matrix = R.from_euler('xyz', eulerangle).as_matrix()
# e_rotation_matrix = np.matmul(rotation_matrix, essential_matrix)
translation = np.asarray(str2list(camera_calib[6]))
fpd = str2list(camera_calib[8])
# resize_fpd = get_cropped_and_resized_intrinsic(fpd[0], fpd[1], fpd[2], fpd[3], 0, 0, 1920/735, 1200/575)
# fpd = resize_fpd
intrinsic = np.asarray([fpd[0], 0, fpd[2], 0,
                        0, fpd[1], fpd[3], 0,
                        0, 0, 1, 0]).reshape(3,4)
distcoeffs = str2list(camera_calib[8])[4:]
# new_intrinsic, _ = cv2.getOptimalNewCameraMatrix(cameraMatrix=intrinsic[:3, :3],
#                                                 distCoeffs=np.asarray(distcoeffs),
#                                                 imageSize=(1200, 1920),
#                                                 alpha=1)
# intrinsic = np.hstack([new_intrinsic, np.asarray([0, 0, 0]).reshape(3, 1)])
extrinsic = np.hstack([rotation_matrix, np.asarray(translation).reshape(3, -1)])
# extrinsic = np.hstack([e_rotation_matrix, np.asarray(translation).reshape(3, -1)])

# projection_matrix = intrinsic
# projection_matrix = np.matmul(intrinsic, extrinsic)
# projection_matrix[2, 2] = 1
# projection_matrix[:, 3] = 0

# p2 = [projection_matrix[0, 0], 0, projection_matrix[0, 2], 0,
#       0, projection_matrix[1, 1], projection_matrix[1, 2], 0,
#       0, 0, 1, 0]
# p2 = projection_matrix.reshape(-1).tolist()
p2 = intrinsic.reshape(-1).tolist()

R0_rect = np.eye(3).reshape(-1).tolist()
# R0_rect = rotation_matrix.reshape(-1).tolist()
Tr_velo_to_cam = extrinsic.reshape(-1).tolist()
# Tr_velo_to_cam = np.zeros((12)).tolist()
Tr_imu_to_velo = np.zeros((12)).tolist()

calib_kitti =  ['P0: '+list2str(p2), 
                'P1: '+list2str(p2), 
                'P2: '+list2str(p2), 
                'P3: '+list2str(p2), 
                'R0_rect: '+list2str(R0_rect), 
                'Tr_velo_to_cam: '+list2str(Tr_velo_to_cam), 
                'Tr_imu_to_velo: '+list2str(Tr_imu_to_velo)]

with open(path + 'test48.txt', 'w') as f:
    f.write('\n'.join(calib_kitti))

# print(essential_matrix, '\n')
# print(rotation_matrix, '\n')
# print(extrinsic, '\n')

# with open('/data/hwang/datasets/kitti/training/calib/000038.txt') as f:
#     calib = f.readlines()
#     extrinsic_ = list(map(float, calib[5].replace('\n', '').split(' ')[1:]))
#     extrinsic_ = np.asarray(extrinsic_).reshape(3, 4)    
#     R0_rect = list(map(float, calib[4].replace('\n', '').split(' ')[1:]))
#     R0_rect = np.asarray(R0_rect).reshape(3, 3)

# print(extrinsic_)

## # label_2
---

In [ ]:
path = '/data/NIA50/50-2/data/NIA48/training/label_2/'
os.makedirs(path, exist_ok=True)


with open('/data/NIA50/50-2/data/NIA48/fine_data/S_Clip_02244_02/result/2-048_02244_FC_001.json', 'r') as f:
    meta = json.load(f)
    
type_ = meta['annotation'][0]['category']
truncation = 0.0
occulsion = 0
xyxy = meta['annotation'][0]['3d_box'][0]['2d_box']
xyz = meta['annotation'][0]['3d_box'][0]['location']
rotation_y = abs(meta['annotation'][0]['3d_box'][0]['rotation_y']) - 90 * np.pi/180

xyz_re = np.matmul(extrinsic, np.asarray(xyz + [1]).reshape(4, 1)).reshape(-1).tolist()
# xyz = np.matmul(e_rotation_matrix, np.asarray(xyz).reshape(3, 1)).reshape(-1).tolist()
whl = meta['annotation'][0]['3d_box'][0]['dimension']
hwlxyz = [whl[1], whl[0], whl[2]] + xyz_re

alpha = cal_alpha_ori(xyz_re[0], xyz_re[2], rotation_y)

xyxy = ' '.join(map(str, xyxy))
hwlxyz = ' '.join(map(str, hwlxyz))

label = list(map(str, [type_, truncation, occulsion, alpha, xyxy, hwlxyz, rotation_y]))

with open(path+'test48.txt', 'w') as f:
    f.write(' '.join(label))

In [ ]:
num = '02244'

path = '/data/NIA50/50-2/data/NIA48/training/calib/'
os.makedirs(path, exist_ok=True)

with open(f'/data/NIA50/50-2/data/NIA48/fine_data/S_Clip_{num}_02/calib/Lidar_camera_calib/2-048_{num}_LCC_CF.txt') as f:
    camera_calib = f.readlines()
    
eulerangle = str2list(camera_calib[4])
eulerangle = np.asarray(eulerangle) + np.asarray([90*np.pi/180, -90*np.pi/180, -180*np.pi/180])
eulerangle = np.asarray(eulerangle) + np.asarray([0*np.pi/180, 0*np.pi/180, 90*np.pi/180])
# eulerangle[2] = eulerangle[2]*-1

X = R.from_euler('X', eulerangle[1]).as_matrix()
Y = R.from_euler('Y', eulerangle[0]).as_matrix()
Z = R.from_euler('Z', eulerangle[2]).as_matrix()
rotation_matrix = Z @ Y @ X

# e_rotation_matrix = np.matmul(essential_matrix, rotation_matrix)
# e_rotation_matrix = np.matmul(rotation_matrix, essential_matrix)
translation = np.asarray(str2list(camera_calib[6]))
# translation = translation = np.asarray(str2list(camera_calib[6])) + np.asarray([0, 0.6, 0.2])
fpd = str2list(camera_calib[8])
# resize_fpd = get_cropped_and_resized_intrinsic(fpd[0], fpd[1], fpd[2], fpd[3], 0, 0, 1920/745, 1200/575)
# fpd = resize_fpd
intrinsic = np.asarray([fpd[0], 0, fpd[2], 0,
                        0, fpd[1], fpd[3], 0,
                        0, 0, 1, 0]).reshape(3,4)
# distortion = str2list(camera_calib[8])[4:]
# new_intrinsic, _ = cv2.getOptimalNewCameraMatrix(cameraMatrix=intrinsic[:3, :3],
#                                                 distCoeffs=np.asarray(distortion),
#                                                 imageSize=(1200, 1920),
#                                                 alpha=1)
# intrinsic = np.hstack([new_intrinsic, np.asarray([0, 0, 0]).reshape(3, 1)])
extrinsic = np.hstack([rotation_matrix, np.asarray(translation).reshape(3, -1)])

p2 = intrinsic.reshape(-1).tolist()

R0_rect = np.eye(3).reshape(-1).tolist()
# R0_rect = rotation_matrix.reshape(-1).tolist()
Tr_velo_to_cam = extrinsic.reshape(-1).tolist()
# Tr_velo_to_cam = np.zeros((12)).tolist()
Tr_imu_to_velo = np.zeros((12)).tolist()

calib_kitti =  ['P0: '+list2str(p2), 
                'P1: '+list2str(p2), 
                'P2: '+list2str(p2), 
                'P3: '+list2str(p2), 
                'R0_rect: '+list2str(R0_rect), 
                'Tr_velo_to_cam: '+list2str(Tr_velo_to_cam), 
                'Tr_imu_to_velo: '+list2str(Tr_imu_to_velo)]

with open(path + 'test48.txt', 'w') as f:
    f.write('\n'.join(calib_kitti))

path = '/data/NIA50/50-2/data/NIA48/training/label_2/'
os.makedirs(path, exist_ok=True)


with open(f'/data/NIA50/50-2/data/NIA48/fine_data/S_Clip_{num}_02/result/2-048_{num}_FC_001.json', 'r') as f:
    meta = json.load(f)
    
type_ = meta['annotation'][0]['category']
truncation = 0.0
occulsion = 0
xyxy = meta['annotation'][0]['3d_box'][0]['2d_box']
xyz = meta['annotation'][0]['3d_box'][0]['location']
rotation_y = np.abs(meta['annotation'][0]['3d_box'][0]['rotation_y']) - 90 * np.pi/180 # radian
# rotation_y = meta['annotation'][0]['3d_box'][0]['rotation_y']


xyz_re = np.matmul(extrinsic, np.asarray(xyz + [1]).reshape(4, 1)).reshape(-1).tolist()
# xyz = np.matmul(e_rotation_matrix, np.asarray(xyz).reshape(3, 1)).reshape(-1).tolist()
whl = meta['annotation'][0]['3d_box'][0]['dimension']
hwlxyz = [whl[1], whl[0], whl[2]] + xyz_re
# hwlxyz = [whl[1], whl[0], whl[2]] + [xyz_re[2], xyz_re[1], xyz_re[0]]

alpha = cal_alpha_ori(xyz_re[0], xyz_re[2], rotation_y)

xyxy = ' '.join(map(str, xyxy))
hwlxyz = ' '.join(map(str, hwlxyz))

label = list(map(str, [type_, truncation, occulsion, alpha, xyxy, hwlxyz, rotation_y]))

with open(path+'test48.txt', 'w') as f:
    f.write(' '.join(label))

In [ ]:
eulerangle = str2list(camera_calib[4])
# eulerangle = np.asarray(eulerangle) + np.asarray([-180*np.pi/180, 90*np.pi/180, -90*np.pi/180])
R.from_euler('XYZ', eulerangle).as_matrix().T

In [ ]:
eulerangle = str2list(camera_calib[4])
# eulerangle = np.asarray(eulerangle) + np.asarray([-180*np.pi/180, 90*np.pi/180, -90*np.pi/180])
R.from_euler('xyz', eulerangle).as_matrix()

In [ ]:
np.asarray(Tr_velo, dtype=float).reshape(-1, 4)

In [ ]:

path = '/data/NIA50/50-2/data/NIA48/training/calib/'
os.makedirs(path, exist_ok=True)

with open('/data/NIA50/50-2/data/NIA48/fine_data/S_Clip_02244_02/calib/Lidar_camera_calib/2-048_02244_LCC_CF.txt') as f:
    camera_calib = f.readlines()
# with open('/data/NIA48/S_Clip_03400_06/calib/Lidar_radar_calib/2-048_03400_LRC_RF.txt') as f:
#     radar_calib = f.readlines()

# essential_matrix = np.asarray([[0, 1, 0],
#                                [0, 0, 1],
#                                [1, 0, 0]])
# essential_matrix = E
eulerangle = str2list(camera_calib[4])
eulerangle = np.asarray(eulerangle) + np.asarray([360*np.pi/180, 90*np.pi/180, 90*np.pi/180])
rotation_matrix = R.from_euler('xyz', eulerangle).as_matrix()
# e_rotation_matrix = np.matmul(essential_matrix, rotation_matrix)
# e_rotation_matrix = np.matmul(rotation_matrix, essential_matrix)
translation = np.asarray(str2list(camera_calib[6]))
fpd = str2list(camera_calib[8])
intrinsic = np.asarray([fpd[0], 0, fpd[2], 0,
                        0, fpd[1], fpd[3], 0,
                        0, 0, 1, 0]).reshape(3,4)
distortion = str2list(camera_calib[8])[4:]
new_intrinsic, _ = cv2.getOptimalNewCameraMatrix(cameraMatrix=intrinsic[:3, :3],
                                                distCoeffs=np.asarray(distortion),
                                                imageSize=(1200, 1920),
                                                alpha=0)
intrinsic = np.hstack([new_intrinsic, np.asarray([0, 0, 0]).reshape(3, 1)])
extrinsic = np.hstack([rotation_matrix, np.asarray(translation).reshape(3, -1)])
# extrinsic = np.hstack([e_rotation_matrix, np.asarray(translation).reshape(3, -1)])

# projection_matrix = intrinsic
# projection_matrix = np.matmul(intrinsic, extrinsic)
# projection_matrix[2, 2] = 1
# projection_matrix[:, 3] = 0

# p2 = [projection_matrix[0, 0], 0, projection_matrix[0, 2], 0,
#       0, projection_matrix[1, 1], projection_matrix[1, 2], 0,
#       0, 0, 1, 0]
# p2 = projection_matrix.reshape(-1).tolist()
p2 = intrinsic.reshape(-1).tolist()

R0_rect = np.eye(3).reshape(-1).tolist()
# R0_rect = rotation_matrix.reshape(-1).tolist()
Tr_velo_to_cam = extrinsic.reshape(-1).tolist()
# Tr_velo_to_cam = np.zeros((12)).tolist()
Tr_imu_to_velo = np.zeros((12)).tolist()

calib_kitti =  ['P0: '+list2str(p2), 
                'P1: '+list2str(p2), 
                'P2: '+list2str(p2), 
                'P3: '+list2str(p2), 
                'R0_rect: '+list2str(R0_rect), 
                'Tr_velo_to_cam: '+list2str(Tr_velo_to_cam), 
                'Tr_imu_to_velo: '+list2str(Tr_imu_to_velo)]

with open(path + 'test48.txt', 'w') as f:
    f.write('\n'.join(calib_kitti))

# print(essential_matrix, '\n')
# print(rotation_matrix, '\n')
# print(extrinsic, '\n')

# with open('/data/hwang/datasets/kitti/training/calib/000038.txt') as f:
#     calib = f.readlines()
#     extrinsic_ = list(map(float, calib[5].replace('\n', '').split(' ')[1:]))
#     extrinsic_ = np.asarray(extrinsic_).reshape(3, 4)    
#     R0_rect = list(map(float, calib[4].replace('\n', '').split(' ')[1:]))
#     R0_rect = np.asarray(R0_rect).reshape(3, 3)

# print(extrinsic_)

path = '/data/NIA50/50-2/data/NIA48/training/label_2/'
os.makedirs(path, exist_ok=True)


with open('/data/NIA50/50-2/data/NIA48/fine_data/S_Clip_02244_02/result/2-048_02244_FC_001.json', 'r') as f:
    meta = json.load(f)
    
type_ = meta['annotation'][0]['category']
truncation = 0.0
occulsion = 0
xyxy = meta['annotation'][0]['3d_box'][0]['2d_box']
xyz = meta['annotation'][0]['3d_box'][0]['location']
rotation_y = abs(meta['annotation'][0]['3d_box'][0]['rotation_y']) - 90 * np.pi/180

xyz_re = np.matmul(extrinsic, np.asarray(xyz + [1]).reshape(4, 1)).reshape(-1).tolist()
# xyz = np.matmul(e_rotation_matrix, np.asarray(xyz).reshape(3, 1)).reshape(-1).tolist()
whl = meta['annotation'][0]['3d_box'][0]['dimension']
hwlxyz = [whl[1], whl[0], whl[2]] + xyz_re

alpha = cal_alpha_ori(xyz_re[0], xyz_re[2], rotation_y)

xyxy = ' '.join(map(str, xyxy))
hwlxyz = ' '.join(map(str, hwlxyz))

label = list(map(str, [type_, truncation, occulsion, alpha, xyxy, hwlxyz, rotation_y]))

with open(path+'test48.txt', 'w') as f:
    f.write(' '.join(label))

# # NIA50
---

## # DeepfusionMOT
---

In [2]:
# Z축 이동을 위해서 calib와 매칭하여 이동범위 지정

calibs = sorted(glob.glob('/data/NIA50/50-2/data/NIA50/nia50_all/raw/*/calib/camera/camera_0.json'))

calib_ls = []
scenes = []
for calib in calibs:
    scene = re.findall('[a-zA-Z0-9_]+', calib)[-5]
    with open(calib, 'r') as f:
        calib = json.load(f)
    if calib['extrinsic'] not in calib_ls:
        calib_ls.append(calib['extrinsic'])
        scenes.append(scene)
    
calib_typ = {'typ1': {'calib': calib_ls[0], 'mov_zpoint': 14},
             'typ2': {'calib': calib_ls[1], 'mov_zpoint': 13},
             'typ3': {'calib': calib_ls[2], 'mov_zpoint': 0},
             'typ4': {'calib': calib_ls[3], 'mov_zpoint': -20}}

In [3]:
# yolov5, pvrcnn inference 완료한 데이터

with open('/data/NIA50/50-2/data/NIA50/nia50_all/pvrcnn_allcat/ImageSets/test.txt', 'r') as f:
    scenes = [re.sub('\n', '', i)[:-5] for i in f.readlines()]
    scenes = sorted(list(set(scenes)))

In [6]:
source_path = '/data/NIA50/50-2/data/NIA50/nia50_all/raw/'
save_path = '/data/NIA50/50-2/data/NIA50/nia50_all/deepfusionmot_allcat/'

### # calib
---

In [ ]:
save_calib = f'{save_path}/calib/'
os.makedirs(save_calib, exist_ok=True)

for scene in scenes:
    calib = sorted(glob.glob(f'{source_path}/{scene}/calib/camera/*.json'))[0]

    with open(calib, 'r') as f:
        calib = json.load(f)

    for typ in ['typ1', 'typ2', 'typ3', 'typ4']:
        if calib['extrinsic'] == calib_typ[typ]['calib']:
            extrinsic = np.asarray(calib['extrinsic']).reshape(4, 4)
            extrinsic[:3, 3] -= extrinsic[:3, 2] * calib_typ[typ]['mov_zpoint']

    p2 = np.hstack([np.asarray(calib['intrinsic']).reshape(3, 3), np.asarray([0, 0, 0]).reshape(3, -1)]).flatten().tolist()
    R0_rect = np.eye(3).reshape(-1).tolist()
    Tr_velo_to_cam = extrinsic.flatten().tolist()[:12]
    Tr_imu_to_velo = np.zeros((12)).tolist()

    calib_kitti =  ['P0: '+list2str(p2), 
                    'P1: '+list2str(p2), 
                    'P2: '+list2str(p2), 
                    'P3: '+list2str(p2), 
                    'R0_rect: '+list2str(R0_rect), 
                    'Tr_velo_to_cam: '+list2str(Tr_velo_to_cam), 
                    'Tr_imu_to_velo: '+list2str(Tr_imu_to_velo)]

    with open(save_calib+f'{scene}.txt', 'w') as f:
        f.write('\n'.join(calib_kitti))

In [ ]:
# save_calib = f'{save_path}/calib/'
# os.makedirs(save_calib, exist_ok=True)

# for scene in scenes:
#     calib = sorted(glob.glob(f'{source_path}/{scene}/calib/camera/*.json'))[0]
# #  = re.search('Suwon_[a-zA-Z0-9_]+', calib).group()
# # # img_path = '/data/NIA50/SUSTechPOINTS_2-050/data/Suwon_A_2210261635_0001_calib_ok/camera/camera_0'
# # # frame = re.search('[0-9]+.pcd', calib).group().replace('.json', '')

#     with open(calib, 'r') as f:
#         calib = json.load(f)

#     p2 = np.hstack([np.asarray(calib['intrinsic']).reshape(3, 3), np.asarray([0, 0, 0]).reshape(3, -1)]).flatten().tolist()
#     R0_rect = np.eye(3).reshape(-1).tolist()
#     Tr_velo_to_cam = calib['extrinsic'][:12]
#     Tr_imu_to_velo = np.zeros((12)).tolist()

#     calib_kitti =  ['P0: '+list2str(p2), 
#                     'P1: '+list2str(p2), 
#                     'P2: '+list2str(p2), 
#                     'P3: '+list2str(p2), 
#                     'R0_rect: '+list2str(R0_rect), 
#                     'Tr_velo_to_cam: '+list2str(Tr_velo_to_cam), 
#                     'Tr_imu_to_velo: '+list2str(Tr_imu_to_velo)]

#     with open(save_calib+f'{scene}.txt', 'w') as f:
#         f.write('\n'.join(calib_kitti))

In [ ]:
# os.makedirs('/data/NIA50/50-2/data/mot_nia50/pointrcnn/training/calib', exist_ok=True)


# path = '/data/NIA50/50-2/data/NIA50_samples/'

# clips = sorted(os.listdir(path))
# for clip in clips[1:2]:
#     frames = sorted(os.listdir(path + clip + '/lidar'))
    
#     for frame in frames:
#         frame = frame[:4]
#         with open(path+f'{clip}/calib_1.txt') as f:
#             calib = f.readlines()

#         # essential_matrix = np.asarray([[0, 1, 0],
#         #                                [0, 0, 1],
#         #                                [1, 0, 0]])
#         # essential_matrix = np.eye(3)
#         eulerangle = str2list(calib[4])
#         X = R.from_euler('X', eulerangle[1]).as_matrix()
#         Y = R.from_euler('Y', eulerangle[0]).as_matrix()
#         Z = R.from_euler('Z', eulerangle[2]).as_matrix()
#         # X = R.from_euler('X', eulerangle[2]).as_matrix()
#         # Y = R.from_euler('Y', eulerangle[1]).as_matrix()
#         # Z = R.from_euler('Z', eulerangle[0]).as_matrix()
#         rotation_matrix = Z @ Y @ X
#         # rotation_matrix = R.from_euler('xyz', eulerangle).as_matrix()
#         # e_rotation_matrix = np.matmul(essential_matrix, rotation_matrix)
#         # e_rotation_matrix = np.matmul(rotation_matrix, essential_matrix)
#         translation = str2list(calib[6])
#         fpd = str2list(calib[8])
#         intrinsic = np.asarray([fpd[0], 0, fpd[2], 0,
#                                 0, fpd[1], fpd[3], 0,
#                                 0, 0, 1, 0]).reshape(3,4)
#         extrinsic = np.hstack([rotation_matrix, np.asarray(translation).reshape(3, -1)])


#         p2 = intrinsic.reshape(-1).tolist()
#         R0_rect = np.eye(3).reshape(-1).tolist()
#         Tr_velo_to_cam = extrinsic.reshape(-1).tolist()
#         Tr_imu_to_velo = np.zeros((12)).tolist()

#         calib_kitti =  ['P0: '+list2str(p2), 
#                         'P1: '+list2str(p2), 
#                         'P2: '+list2str(p2), 
#                         'P3: '+list2str(p2), 
#                         'R0_rect: '+list2str(R0_rect), 
#                         'Tr_velo_to_cam: '+list2str(Tr_velo_to_cam), 
#                         'Tr_imu_to_velo: '+list2str(Tr_imu_to_velo)]

#         with open(f'/data/NIA50/50-2/data/mot_nia50/pointrcnn/training/calib/{clip}_{frame}.txt', 'w') as f:
#             f.write('\n'.join(calib_kitti))

In [ ]:
# calib_json = {
#     "extrinsic": np.asarray(np.vstack([extrinsic, np.asarray([0, 0, 0, 1])]), dtype=np.float32).flatten().tolist(),
#     "intrinsic": np.asarray(p2, dtype=np.float32).reshape(-1, 4)[:3, :3].flatten().tolist()
# }
# json_object = json.dumps(calib_json, indent = 4) 
# # Writing to sample.json 
# with open('/data/NIA50/SUSTechPOINTS_2-050/data/Suwon_A_0000/calib/camera/front.json', "w") as outfile: 
#     outfile.write(json_object) 

### # 2D_yolov5
---
- = 2D_rrc_Car_train

In [ ]:
save_2d_label = f'{save_path}/2D_yolov5/'
os.makedirs(save_2d_label, exist_ok=True)

w = 1920
h = 1200
for scene in scenes:
    labels = sorted(glob.glob(f'/NIA50/50-2/result/yolov5/test/labels/{scene}*.txt'))

    label_df = pd.DataFrame()
    for label in labels:
        # with open(label, 'r') as f:
        #     bbox = [re.sub('\n', '', j) for j in f.readlines()]
        frame_df = pd.read_csv(label, header=None, sep=' ')
        frame_df.columns = ['frame', 'x', 'y', 'w', 'h', 'conf']
        frame_df['frame'] = int(re.findall('[0-9]+', label)[-1])
        frame_df['xmin'] = (frame_df['x'] - frame_df['w']/2) * w
        frame_df['ymin'] = (frame_df['y'] - frame_df['h']/2) * h
        frame_df['xmax'] = (frame_df['x'] + frame_df['w']/2) * w
        frame_df['ymax'] = (frame_df['y'] + frame_df['h']/2) * h

        label_df = pd.concat((label_df, frame_df[['frame', 'xmin', 'ymin', 'xmax', 'ymax', 'conf']]), axis=0)
    label_df.to_csv(save_2d_label+f'{scene}.txt', index=None, header=None, sep=',')

In [ ]:
# save_2d_label = f'{save_path}/2D_yolov5/'
# os.makedirs(save_2d_label, exist_ok=True)

# w = 1920
# h = 1200
# for scene in scenes:
#     labels = sorted(glob.glob(f'/data/NIA50/50-2/models/yolov5/runs/nia50_train1st/{scene}/labels/*.txt'))

#     label_df = pd.DataFrame()
#     for i, label in enumerate(labels):
#         # with open(label, 'r') as f:
#         #     bbox = [re.sub('\n', '', j) for j in f.readlines()]
#         frame_df = pd.read_csv(label, header=None, sep=' ')
#         frame_df[0] = i
#         frame_df.columns = ['frame', 'x', 'y', 'w', 'h', 'conf']
#         frame_df['xmin'] = (frame_df['x'] - frame_df['w']/2) * w
#         frame_df['ymin'] = (frame_df['y'] - frame_df['h']/2) * h
#         frame_df['xmax'] = (frame_df['x'] + frame_df['w']/2) * w
#         frame_df['ymax'] = (frame_df['y'] + frame_df['h']/2) * h

#         label_df = pd.concat((label_df, frame_df[['frame', 'xmin', 'ymin', 'xmax', 'ymax', 'conf']]), axis=0)
#     label_df.to_csv(save_2d_label+f'{scene}.txt', index=None, header=None, sep=',')

In [ ]:
# def xywhn2xyxy(x, w=1920, h=1200):
#     # Convert nx4 boxes from [x, y, w, h] to [x1, y1, x2, y2] where xy1=top-left, xy2=bottom-right
#     x = np.array(x).reshape(1, -1)
#     y = np.copy(x)
#     y[:, 0] = (x[:, 0] - x[:, 2] / 2) * w  # top left x
#     y[:, 1] = (x[:, 1] - x[:, 3] / 2) * h  # top left y
#     y[:, 2] = (x[:, 0] + x[:, 2] / 2) * w  # bottom right x
#     y[:, 3] = (x[:, 1] + x[:, 3] / 2) * h  # bottom right y
#     y = list(np.around(y).astype(int).reshape(-1))
#     return y

### # 3D_pvrcnn
---
- = 3D_pointrcnn_Car_Val

In [ ]:
# 문자열 숫자리스트로 바꾸는 함수
def str2list(txt):
    txt = txt.replace('\n', '').split(' ')[1:]
    txt = list(map(float, txt))
    
    return txt


# 리스트를 문자열로 바꾸는 함수
def list2str(list):
    list = ' '.join(map(str, list))
    
    return list


# alpha 구하는 공식
import math

def normalizeAngle(angle):
    result = angle % (2*math.pi)
    if result < -math.pi:
        result += 2*math.pi
    elif result > math.pi:
        result -= 2*math.pi
    return result

def cal_alpha_ori(x, z, ry):  
    angle = ry
    angle -= -math.atan2(z, x) -1.5*math.pi 
    alpha = normalizeAngle(angle)
    return alpha # -1.818032754845337


def roty(t, Rx=90/180*np.pi):
    ''' Rotation about the y-axis. '''
    c = np.cos(t)
    s = np.sin(t)
    # return  np.array([[c, 0, s],
    #                 [0, 1, 0],
    #                 [-s, 0, c]])

    X = np.array([[1, 0, 0],
                    [0, np.cos(Rx), -np.sin(Rx)],
                    [0, np.sin(Rx), np.cos(Rx)]])

    # X = np.eye(3)

    Z = np.array([[c, -s, 0],
                    [s, c, 0],
                    [0, 0, 1]])
    
    return np.matmul(Z, X)

In [15]:
save_3d_label = save_path + '/3D_pvrcnn/'
os.makedirs(save_3d_label, exist_ok=True)

with open('/data/NIA50/50-2/result/pvrcnn_allcat/test/eval/result.pkl', 'rb') as f:
    results = pkl.load(f)

for scene in scenes:

    label_df = pd.DataFrame()
    for result in results:
        frame_df = pd.DataFrame(columns=['frame_id', 'type', 'x1', 'y1', 'x2', 'y2', 'score', 'h', 'w', 'l', 'x', 'y', 'z', 'rot_y'])
        if scene in result['frame_id']:
            frame_df['type'] = result['pred_labels']
            frame_df['score'] = result['score']
            frame_df['h'] = result['boxes_lidar'][:, 5]
            frame_df['w'] = result['boxes_lidar'][:, 4]
            frame_df['l'] = result['boxes_lidar'][:, 3]
            frame_df['x'] = result['boxes_lidar'][:, 0]
            frame_df['y'] = result['boxes_lidar'][:, 1]
            frame_df['z'] = result['boxes_lidar'][:, 2]
            frame_df['rot_y'] = result['boxes_lidar'][:, 6]
            frame_df['frame_id'] = result['frame_id']

        label_df = pd.concat([label_df, frame_df], axis=0)

    label_df['scene'] = label_df['frame_id'].apply(lambda x: x[:-5])
    label_df['frame'] = label_df['frame_id'].apply(lambda x: int(x[-4:]))
    # label_df = label_df.loc[(-75.2 < label_df['x']) & (label_df['x'] < 75.2) & (-150.4 < label_df['y']) & (label_df['y'] < 0) & (-4 < label_df['z']) & (label_df['z'] < 8)]

    with open(save_path+f'calib/{scene}.txt') as f:
        calib = [re.sub('\n', '', i) for i in f.readlines()]
    intrinsic = np.asarray(calib[2].split(' ')[1:], dtype=float).reshape(3, 4)
    extrinsic = np.asarray(calib[5].split(' ')[1:], dtype=float).reshape(3, 4)
    extrinsic = np.vstack([extrinsic, [0, 0, 0, 1]])
    # scene_df = label_df.loc[label_df['scene']==scene]

    minx = []
    miny = []
    maxx = []
    maxy = []
    for i in np.arange(len(label_df)):
        obj = label_df.iloc[i]

        R = roty(obj['rot_y'])
        x = obj['x']
        y = obj['y']
        z = obj['z']
        l = obj['l']
        w = obj['w']
        h = obj['h']
        
        x_corners = [l / 2, l / 2, -l / 2, -l / 2, l / 2, l / 2, -l / 2, -l / 2];
        y_corners = [h / 2, h / 2, h / 2, h / 2, -h / 2, -h / 2, -h / 2, -h / 2];
        z_corners = [w / 2, -w / 2, -w / 2, w / 2, w / 2, -w / 2, -w / 2, w / 2];
        
        corners_3d = np.dot(R, np.vstack([x_corners, y_corners, z_corners]))
        corners_3d[0, :] = corners_3d[0, :] + x  # x
        corners_3d[1, :] = corners_3d[1, :] + y  # y
        corners_3d[2, :] = corners_3d[2, :] + z  # z
        corners_3d = np.vstack([corners_3d, [1, 1, 1, 1, 1, 1, 1, 1]])
        
        point2d = np.matmul(intrinsic, np.matmul(extrinsic, corners_3d))
        pointx = np.around(point2d/point2d[2])[0]
        pointy = np.around(point2d/point2d[2])[1]
        pointx[pointx > 1920] = 1920
        pointx[pointx < 0] = 0
        pointy[pointy > 1200] = 1200
        pointy[pointy < 0] = 0

        minx.append(min(pointx))
        miny.append(min(pointy))
        maxx.append(max(pointx))
        maxy.append(max(pointy))

    label_df[['x1', 'y1', 'x2', 'y2']] = np.asarray((minx, miny, maxx, maxy)).T
    label_df['alpha'] = 0
    # label_df = label_df.loc[(label_df['x2']-label_df['x1']!=0) & (label_df['y2']-label_df['y1']!=0)]
    label_df = label_df[['frame', 'type', 'x1', 'y1', 'x2', 'y2', 'score', 'h', 'w', 'l', 'x', 'y', 'z', 'rot_y', 'alpha']]
    label_df.to_csv(save_3d_label+f'{scene}.txt', index=None, header=None, sep=',')

    # scene_df[['x1', 'y1', 'x2', 'y2']] = np.asarray((minx, miny, maxx, maxy)).T
    # scene_df['alpha'] = 0
    # scene_df = scene_df[['frame', 'type', 'x1', 'y1', 'x2', 'y2', 'score', 'h', 'w', 'l', 'x', 'y', 'z', 'rot_y', 'alpha']]
    # scene_df.to_csv(save_3d_label+f'{scene}.txt', index=None, header=None, sep=',')

In [ ]:
# save_3d_label = save_path + '/3D_pvrcnn/'
# os.makedirs(save_3d_label, exist_ok=True)

# with open('/data/NIA50/50-2/models/OpenPCD/output/nia50_train1st_pvrcnn/result.pkl', 'rb') as f:
#     results = pkl.load(f)

# dev = (len(results)//1000) + 1
# for idx in np.arange(dev):
#     start = int(idx*1000)
#     end = int((idx+1)*1000)

#     label_df = pd.DataFrame()
#     for result in results[start:end]:
#         frame_df = pd.DataFrame(columns=['frame_id', 'type', 'x1', 'y1', 'x2', 'y2', 'score', 'h', 'w', 'l', 'x', 'y', 'z', 'rot_y'])
#         frame_df['type'] = result['pred_labels']
#         frame_df['score'] = result['score']
#         frame_df['h'] = result['boxes_lidar'][:, 5]
#         frame_df['w'] = result['boxes_lidar'][:, 4]
#         frame_df['l'] = result['boxes_lidar'][:, 3]
#         frame_df['x'] = result['boxes_lidar'][:, 1]
#         frame_df['y'] = result['boxes_lidar'][:, 0] * -1
#         frame_df['z'] = result['boxes_lidar'][:, 2]
#         frame_df['rot_y'] = result['boxes_lidar'][:, 6]
#         frame_df['frame_id'] = result['frame_id']

#         label_df = pd.concat([label_df, frame_df], axis=0)

#     label_df['scene'] = label_df['frame_id'].apply(lambda x: x[:-5])
#     label_df['frame'] = label_df['frame_id'].apply(lambda x: int(x[-4:]))

#     for scene in label_df['scene'].unique():
#         with open(save_path+f'calib/{scene}.txt') as f:
#             calib = [re.sub('\n', '', i) for i in f.readlines()]
#         intrinsic = np.asarray(calib[2].split(' ')[1:], dtype=float).reshape(3, 4)
#         extrinsic = np.asarray(calib[5].split(' ')[1:], dtype=float).reshape(4, 4)
#         scene_df = label_df.loc[label_df['scene']==scene]

#         minx = []
#         miny = []
#         maxx = []
#         maxy = []
#         for i in np.arange(len(scene_df)):
#             obj = scene_df.iloc[i]

#             R = roty(obj['rot_y'])
#             x = obj['x']
#             y = obj['y']
#             z = obj['z']
#             l = obj['l']
#             w = obj['w']
#             h = obj['h']
            
#             x_corners = [l / 2, l / 2, -l / 2, -l / 2, l / 2, l / 2, -l / 2, -l / 2];
#             y_corners = [h / 2, h / 2, h / 2, h / 2, -h / 2, -h / 2, -h / 2, -h / 2];
#             z_corners = [w / 2, -w / 2, -w / 2, w / 2, w / 2, -w / 2, -w / 2, w / 2];
            
#             corners_3d = np.dot(R, np.vstack([x_corners, y_corners, z_corners]))
#             corners_3d[0, :] = corners_3d[0, :] + x  # x
#             corners_3d[1, :] = corners_3d[1, :] + y  # y
#             corners_3d[2, :] = corners_3d[2, :] + z  # z
#             corners_3d = np.vstack([corners_3d, [1, 1, 1, 1, 1, 1, 1, 1]])
            
#             point2d = np.matmul(intrinsic, np.matmul(extrinsic, corners_3d))
#             pointx = np.around(point2d/point2d[2])[0]
#             pointy = np.around(point2d/point2d[2])[1]
#             pointx[pointx > 1920] = 1920
#             pointx[pointx < 0] = 0
#             pointy[pointy > 1200] = 1200
#             pointy[pointy < 0] = 0

#             minx.append(min(pointx))
#             miny.append(min(pointy))
#             maxx.append(max(pointx))
#             maxy.append(max(pointy))

#         scene_df[['x1', 'y1', 'x2', 'y2']] = np.asarray((minx, miny, maxx, maxy)).T
#         scene_df['alpha'] = 0
#         scene_df = scene_df[['frame', 'type', 'x1', 'y1', 'x2', 'y2', 'score', 'h', 'w', 'l', 'x', 'y', 'z', 'rot_y', 'alpha']]
#         scene_df.to_csv(save_3d_label+f'{scene}.txt', index=None, header=None, sep=',')

### # image_02
---

In [8]:
save_image = save_path + '/image_02/'
os.makedirs(save_path, exist_ok=True)

for scene in scenes:
    img_src = source_path+scene+'/camera/camera_0'
    shutil.copytree(img_src, save_image+scene, dirs_exist_ok=True)

### # velodyne
---
- 필요 없음

In [ ]:
# save_velo = save_path + 'velodyne'
# os.makedirs(save_velo, exist_ok=True)

# for scene in scenes:
#     os.makedirs(save_velo+'/'+scene, exist_ok=True)
#     velodynes = sorted(glob.glob(source_path+scene+'/lidar/*.pcd'))
#     for velodyne in velodynes:
#         # clip = re.search('Suwon_[a-zA-Z0-9_]+', velodyne).group()
#         # clip = re.findall('[a-zA-Z0-9_]+', velodyne)[-4]
#         frame = re.search('[0-9]+.pcd', velodyne).group().replace('.pcd', '')
        
#         pcd = o3d.t.io.read_point_cloud(velodyne)
#         positions = pcd.point.positions.numpy()
#         intensity = pcd.point.intensity.numpy()
#         pcd = np.concatenate((positions, intensity), axis = 1)
#         pcd_bytes = pcd.tobytes()
        
#         with open(save_velo+f'/{scene}/{frame}.bin', 'wb') as f:
#             f.write(pcd_bytes)

In [ ]:
# os.makedirs('/data/NIA50/50-2/data/mot_nia50/pointrcnn/training/velodyne', exist_ok=True)

# path = '/data/NIA50/50-2/data/NIA50_samples/'
# clips = sorted(os.listdir(path))
# for clip in clips[1:2]:
#     # frames = sorted(os.listdir(f'/data/NIA50/50-2/data/NIA50_samples/{clip}/lidar'))    
#     pcd_files = sorted(glob.glob(path+f'{clip}/lidar/*.pcd'))
#     for pcd_file in pcd_files:
#         frame = pcd_file[-8:-4]
#         pcd = o3d.t.io.read_point_cloud(pcd_file)
#         positions = pcd.point.positions.numpy()
#         intensity = pcd.point.intensity.numpy()
        
#         pcd = np.concatenate((positions, intensity), axis = 1)
#         pcd_bytes = pcd.tobytes()
        
#         with open(f'/data/NIA50/50-2/data/mot_nia50/pointrcnn/training/velodyne/{clip}_{frame}.bin', 'wb') as f:
#             f.write(pcd_bytes)

In [ ]:
# f'/data/NIA50/50-2/data/mot_nia50/pointrcnn/training/velodyne/{clip}_{frame}.bin'

## # PointRCNN
---

In [ ]:
source_path = '/data/NIA50/50-2/data/NIA50/train_1st/raw/*/'
source_save_path = '/data/NIA50/50-2/data/NIA50/train_1st/pointrcnn/training/'

### # velodyne
---

In [ ]:
save_path = f'{source_save_path}/velodyne/'
os.makedirs(save_path, exist_ok=True)

velodynes = sorted(glob.glob(f'{source_path}lidar/*.pcd'))

for velodyne in velodynes:
    # clip = re.search('Suwon_[a-zA-Z0-9_]+', velodyne).group()
    clip = re.findall('[a-zA-Z0-9_]+', velodyne)[-4]
    frame = re.search('[0-9]+.pcd', velodyne).group().replace('.pcd', '')
    
    pcd = o3d.t.io.read_point_cloud(velodyne)
    positions = pcd.point.positions.numpy()
    intensity = pcd.point.intensity.numpy()
    pcd = np.concatenate((positions, intensity), axis = 1)
    pcd_bytes = pcd.tobytes()
    
    # os.makedirs(save_path+clip, exist_ok=True)
    with open(save_path+f'{clip}_{frame}.bin', 'wb') as f:
        f.write(pcd_bytes)

### # calib
---

In [ ]:
save_path = f'{source_save_path}/calib/'
os.makedirs(save_path, exist_ok=True)

calibs = sorted(glob.glob(f'{source_path}/calib/camera/*.json'))
frames = [str(i).zfill(4) for i in np.arange(10).tolist()]

for calib in calibs:
    # clip = re.search('Suwon_[a-zA-Z0-9_]+', calib).group()
    scene = re.findall('[a-zA-Z0-9_]+', calib)[-5]
    # img_path = '/data/NIA50/SUSTechPOINTS_2-050/data/Suwon_A_2210261635_0001_calib_ok/camera/camera_0'
    # frame = re.search('[0-9]+.pcd', calib).group().replace('.json', '')

    with open(calib, 'r') as f:
        calib = json.load(f)

    p2 = np.hstack([np.asarray(calib['intrinsic']).reshape(3, 3), np.asarray([0, 0, 0]).reshape(3, -1)]).flatten().tolist()
    R0_rect = np.eye(3).reshape(-1).tolist()
    # viewmatrix = np.asarray([0, -1, 0,
    #                          0, 0, -1,
    #                          -1, 0, 0]).reshape(3, 3)
    # extrinsic = np.asarray(calib['extrinsic']).reshape(4, 4)
    # proj_matrix = np.matmul(viewmatrix, extrinsic[:3, :3])
    # extrinsic[:3, :3] = proj_matrix
    # Tr_velo_to_cam = extrinsic.reshape(-1).tolist()[:12]
    # Tr_velo_to_cam = proj_matrix.reshape(-1).tolist()[:12]
    Tr_velo_to_cam = calib['extrinsic'][:12]
    Tr_imu_to_velo = np.zeros((12)).tolist()

    calib_kitti =  ['P0: '+list2str(p2), 
                    'P1: '+list2str(p2), 
                    'P2: '+list2str(p2), 
                    'P3: '+list2str(p2), 
                    'R0_rect: '+list2str(R0_rect), 
                    'Tr_velo_to_cam: '+list2str(Tr_velo_to_cam), 
                    'Tr_imu_to_velo: '+list2str(Tr_imu_to_velo)]

    for frame in frames:
        with open(save_path+f'{scene}_{frame}.txt', 'w') as f:
            f.write('\n'.join(calib_kitti))

### # label_2
---

In [ ]:
def roty(t, Rx=90/180*np.pi):
    ''' Rotation about the y-axis. '''
    c = np.cos(t)
    s = np.sin(t)
    # return  np.array([[c, 0, s],
    #                 [0, 1, 0],
    #                 [-s, 0, c]])

    X = np.array([[1, 0, 0],
                    [0, np.cos(Rx), -np.sin(Rx)],
                    [0, np.sin(Rx), np.cos(Rx)]])

    Z = np.array([[c, -s, 0],
                    [s, c, 0],
                    [0, 0, 1]])
    
    return np.matmul(Z, X)


# 문자열 숫자리스트로 바꾸는 함수
def str2list(txt):
    txt = txt.replace('\n', '').split(' ')[1:]
    txt = list(map(float, txt))
    
    return txt


# 리스트를 문자열로 바꾸는 함수
def list2str(list):
    list = ' '.join(map(str, list))
    
    return list


# alpha 구하는 공식
import math

def normalizeAngle(angle):
    result = angle % (2*math.pi)
    if result < -math.pi:
        result += 2*math.pi
    elif result > math.pi:
        result -= 2*math.pi
    return result

def cal_alpha_ori(x, z, ry):  
    angle = ry
    angle -= -math.atan2(z, x) -1.5*math.pi 
    alpha = normalizeAngle(angle)
    return alpha # -1.818032754845337


def xyxy2xywhn(x, w=1920, h=1200, clip=False, eps=0.0):
    # Convert nx4 boxes from [x1, y1, x2, y2] to [x, y, w, h] normalized where xy1=top-left, xy2=bottom-right
    if clip:
        clip_boxes(x, (h - eps, w - eps))  # warning: inplace clip
    x = np.array(x).reshape(1, -1)
    y = np.copy(x)
    y[:, 0] = ((x[:, 0] + x[:, 2]) / 2) / w  # x center
    y[:, 1] = ((x[:, 1] + x[:, 3]) / 2) / h  # y center
    y[:, 2] = (x[:, 2] - x[:, 0]) / w  # width
    y[:, 3] = (x[:, 3] - x[:, 1]) / h  # height
    y = list(y.reshape(-1))
    return y

In [ ]:
save_path = f'{source_save_path}/label_2/'
calib_path = f'{source_save_path}/calib/'
os.makedirs(save_path, exist_ok=True)

type_list = []
rename = {'Adult': 'Pedestrian', 
          'Bus': 'Car',
          'Car': 'Car',
          'Kid': 'Pedestrian',
          'Kickboard': 'Cyclist',
          'Large_Truck': 'Car',
          'Light_Car': 'Car',
          'Medium_Truck': 'Car',
          'Mini_Bus': 'Car',
        #   'Motorcycle': 'MOTORCYCLE',
        #   'Pedestrian': 'Pedestrian',
          'SUV': 'Car',
          'Small_Car': 'Car',
          'Small_Truck': 'Car',
          'Special_Vehicle': 'Car',
          'Two_Wheeler': 'Cyclist',
          'Van': 'Car'}

labels = sorted(glob.glob(f'{source_path}/label/[0-9]*.json'))

for label in labels:
    try:
        # clip = re.search('Suwon_[a-zA-Z0-9_]+', label).group()
        clip = re.findall('[a-zA-Z0-9_]+', label)[-4]

        frame = re.search('[0-9]+.json', label).group().replace('.json', '')

        with open(calib_path+f'{clip}_{frame}.txt', 'r') as f:
            calib = f.readlines()
            # calib = [re.sub('\n', '', i) for i in calib]

        # extrinsic = np.asarray(str2list(calib[5])).reshape(4, 4)[:3, :4]
        intrinsic = np.asarray(str2list(calib[2])).reshape(3, 4)[:3, :3]
        extrinsic = np.asarray(str2list(calib[5])).reshape(3, 4)
        
        with open(label, 'r') as f:
            label = json.load(f)
        
        label_txt = []
        for i in np.arange(len(label)):
            try:
                # frame_num = [int(frame)]
                # id_ = [label[i]['obj_id']]
                type_ = label[i]['obj_type']
                rename_type = [rename[type_]]
                truncation = [0.0]
                occuluded = [0]
                alpha = 0
                # bbox = [0, 0, 0, 0]
                lwh = list(label[i]['psr']['scale'].values())
                dimensions = [lwh[2], lwh[1], lwh[0]]
                xyz = list(label[i]['psr']['position'].values())
                location = np.matmul(extrinsic, np.asarray((xyz + [0])).reshape(4, -1)).flatten().tolist()
                rotation_y = [label[i]['psr']['rotation']['z']]

                # make 2dbox labeling
                l = lwh[0]
                w = lwh[1]
                h = lwh[2]

                x_corners = [l / 2, l / 2, -l / 2, -l / 2, l / 2, l / 2, -l / 2, -l / 2];
                y_corners = [h / 2, h / 2, h / 2, h / 2, -h / 2, -h / 2, -h / 2, -h / 2];
                z_corners = [w / 2, -w / 2, -w / 2, w / 2, w / 2, -w / 2, -w / 2, w / 2];

                R = roty(rotation_y[0])
                corners_3d = np.dot(R, np.vstack([x_corners, y_corners, z_corners]))
                corners_3d[0, :] = corners_3d[0, :] + xyz[0]  # x
                corners_3d[1, :] = corners_3d[1, :] + xyz[1]  # y
                corners_3d[2, :] = corners_3d[2, :] + xyz[2]  # z
                corners_3d = np.vstack([corners_3d, [1, 1, 1, 1, 1, 1, 1, 1]])

                point2d = np.matmul(intrinsic, np.matmul(extrinsic, corners_3d))
                pointx = np.around(point2d/point2d[2])[0]
                pointx[pointx > 1920] = 1920
                pointx[pointx < 0] = 0
                pointy = np.around(point2d/point2d[2])[1]
                pointy[pointy > 1200] = 1200
                pointy[pointy < 0] = 0
                bbox = [min(pointx), min(pointy), max(pointx), max(pointy)]
                # bbox = xyxy2xywhn(box)

                alpha = [cal_alpha_ori(location[0], location[2], rotation_y[0])]
                
                # label_format = list2str(rename_type + truncation + occuluded + alpha + bbox + dimensions + xyz + rotation_y)
                label_format = list2str(rename_type + truncation + occuluded + alpha + bbox + dimensions + location + rotation_y)
                label_txt.append(label_format)
                
                type_list.append(rename_type[0])

            except KeyError:
                print('라벨링 에러 :', clip, frame, type_)
            
            except TypeError:
                print('타입 에러 : ', clip, frame, type_)

        # os.makedirs(save_path+clip, exist_ok=True)
        with open(save_path+f'/{clip}_{frame}.txt', 'w') as f:
            f.write('\n'.join(label_txt))
        
    except ValueError:
        print('json 에러 :', clip, frame)

    except FileNotFoundError:
        print('calib 에러 :', clip, frame)

### # image_02
---

In [7]:
save_path = f'{source_save_path}/image_2/'
os.makedirs(save_path, exist_ok=True)

img_ls = sorted(glob.glob(f'{source_path}/camera/camera_0/*.jpg'))

for img in img_ls:
    clip = re.findall('[a-zA-Z0-9_]+', img)[-5]
    # clip = re.search('Suwon_[a-zA-Z0-9_]+', img).group()
    
    frame = re.search('[0-9]+.jpg', img).group().replace('.jpg', '')

    shutil.copy(img, save_path+f'{clip}_{frame}.jpg')

NameError: name 'source_save_path' is not defined

### # ImageSets
---

In [ ]:
# 빈 라벨 걸러내기
import os

labels = sorted(glob.glob('/data/NIA50/50-2/data/NIA50/train_1st/pointrcnn/training/label_2/*.txt'))

empty_labels = []
for label in labels:
    with open(label, 'r') as f:
        labeled = f.readlines()
    
    if len(labeled) == 0:
        # print(label)
        empty_labels.append(re.findall('[a-zA-Z0-9_]+', label)[-2])

In [ ]:
empty_labels

In [ ]:
with open('/data/NIA50/50-2/data/NIA50/train_1st/yolov5/ImageSets/train.txt', 'r') as f:
    train = f.readlines()
with open('/data/NIA50/50-2/data/NIA50/train_1st/yolov5/ImageSets/val.txt', 'r') as f:
    val = f.readlines()


train = sorted([re.findall('[a-zA-Z0-9_]+', i)[-2] for i in train])
train = [i for i in train if i not in empty_labels]
val = sorted([re.findall('[a-zA-Z0-9_]+', i)[-2] for i in val])
val = [i for i in val if i not in empty_labels]

In [ ]:
save_path = '/data/NIA50/50-2/data/NIA50/train_1st/pointrcnn/ImageSets/'
os.makedirs(save_path, exist_ok=True)

# clips = sorted([i.replace('.txt', '') for i in os.listdir('/data/NIA50/50-2/data/mot_nia50/pointrcnn/training/calib')])

with open(save_path+'train.txt', 'w') as f:
    f.write('\n'.join(train))

with open(save_path+'val.txt', 'w') as f:
    f.write('\n'.join(val))

### # kitti_infos_test.pkl
---

In [ ]:
path = '/data/NIA50/50-2/data/mot_nia50/pointrcnn/testing/'
clips = sorted(os.listdir(path+'image_2'))

test_infos = []
for clip in clips:
    img_ls = sorted(glob.glob(path+f'image_2/{clip}/*.jpg'))

    with open(path+f'/calib/{clip}.txt') as f:
        calib = f.readlines()

    P2 = np.asarray(str2list(calib[2])).reshape(3, 4)
    P2 = np.vstack([P2, np.asarray([0, 0, 0, 1])])
    R0_rect = np.eye(4)
    Tr_velo_to_cam = np.asarray(str2list(calib[5])).reshape(4, 4)

    for img in img_ls:
        idx = re.search('Suwon[a-zA-Z0-9_/]+', img).group()
        img_shape = np.asarray(cv2.imread(img).shape[:2], dtype=np.int32)

        info = {'point_cloud': {'num_features': 4, 'lidar_idx': idx},
         'image': {'image_idx': idx, 'image_shape': img_shape},
         'calib': {'P2': P2,
         'R0_rect': R0_rect,
         'Tr_velo_to_cam': Tr_velo_to_cam}}

        test_infos.append(info)

In [ ]:
with open('/data/NIA50/50-2/data/mot_nia50/pointrcnn/kitti_infos_test.pkl', 'wb') as f:
    pkl.dump(test_infos, f)

In [ ]:
np.asarray(cv2.imread(img).shape[:2], dtype=np.int32)

## # PV-RCNN
---

In [2]:
# Z축 이동을 위해서 calib와 매칭하여 이동범위 지정

src_path = '/data/NIA50/50-2/data/NIA50/nia50_all/raw'
calibs = sorted(glob.glob(f'{src_path}/*/calib/camera/camera_0.json'))

calib_ls = []
scenes = []
for calib in calibs:
    scene = re.findall('[a-zA-Z0-9_]+', calib)[-5]
    with open(calib, 'r') as f:
        calib = json.load(f)
    if calib['extrinsic'] not in calib_ls:
        calib_ls.append(calib['extrinsic'])
        scenes.append(scene)
    
calib_typ = {'typ1': {'calib': calib_ls[0], 'mov_zpoint': 14},
             'typ2': {'calib': calib_ls[1], 'mov_zpoint': 13},
             'typ3': {'calib': calib_ls[2], 'mov_zpoint': 0},
             'typ4': {'calib': calib_ls[3], 'mov_zpoint': -20}}

### # points
---

In [5]:
src_path = '/data/NIA50/50-2/data/NIA50/nia50_all/raw/'
save_path = '/data/NIA50/50-2/data/NIA50/nia50_all/pvrcnn_integ/points/'
os.makedirs(save_path, exist_ok=True)

points = sorted(glob.glob(f'{src_path}/*/lidar/*.pcd'))

for point in points:
    scene = re.findall('[a-zA-Z_0-9_]+', point)[-4]
    frame = re.findall('[0-9]+', point)[-1]
    
    with open(f'{src_path}/{scene}/calib/camera/camera_0.json', 'r') as f:
        calib = json.load(f)
    
    pcd = o3d.t.io.read_point_cloud(point)
    positions = pcd.point.positions.numpy()
    intensity = pcd.point.intensity.numpy()

    if calib['extrinsic'] == calib_typ['typ1']['calib']: positions[:, 2] += calib_typ['typ1']['mov_zpoint']
    elif calib['extrinsic'] == calib_typ['typ2']['calib']: positions[:, 2] += calib_typ['typ2']['mov_zpoint']
    elif calib['extrinsic'] == calib_typ['typ3']['calib']: positions[:, 2] += calib_typ['typ3']['mov_zpoint']
    elif calib['extrinsic'] == calib_typ['typ4']['calib']: positions[:, 2] += calib_typ['typ4']['mov_zpoint']

    pcd = np.concatenate((positions, intensity), axis = 1)

    np.save(save_path+f'/{scene}_{frame}.npy', pcd)

In [ ]:
# points = sorted(glob.glob('/data/NIA50/50-2/data/NIA50_real/*/lidar/*.pcd'))
# save_path = '/data/NIA50/50-2/data/mot_nia50/pvrcnn/points/'

# for point in points:
#     clip = re.search('Suwon_[A-Z_0-9]+', point).group()
#     frame = re.search('[0-9]+.pcd', point).group().replace('.pcd', '')
    
#     pcd = o3d.t.io.read_point_cloud(point)
#     positions = pcd.point.positions.numpy()
#     intensity = pcd.point.intensity.numpy()
#     pcd = np.concatenate((positions, intensity), axis = 1)

#     # npy = np.save(save_path+f'/{clip}_{frame}.npy', pcd)

### # labels
---

In [6]:
src_path = '/data/NIA50/50-2/data/NIA50/nia50_all/raw/'
save_path = '/data/NIA50/50-2/data/NIA50/nia50_all/pvrcnn_integ/labels/'
os.makedirs(save_path, exist_ok=True)

change_class_ls = {'Small_Car': 'Car',
                    'Light_Car': 'Car',
                    'Car': 'Car',
                    'Van': 'Van',
                    'SUV': 'SUV',
                    'Small_Truck': 'Truck',
                    'Medium_Truck': 'Truck',
                    'Large_Truck': 'Truck',
                    'Mini_Bus': 'Bus',
                    'Bus': 'Bus',
                    'Special_Vehicle': 'Special_Vehicle',
                    'Two_Wheeler': 'Two_Wheeler',
                    'Kickboard': 'Two_Wheeler',
                    'Adult': 'Person',
                    'Kid': 'Person'}

labels = sorted(glob.glob(src_path+'*/label/*.json'))

x = []
y = []
z = []
for label in labels:
    scene = re.findall('[a-zA-Z_0-9_]+', label)[-4]
    frame = re.findall('[0-9]+', label)[-1]

    with open(f'{src_path}/{scene}/calib/camera/camera_0.json', 'r') as f:
        calib = json.load(f)
    
    with open(label, 'r') as f:
        label = json.load(f)
    
    labeling_ls = []
    for i in np.arange(len(label)):
        try:
            xyz = list(label[i]['psr']['position'].values())

            if calib['extrinsic'] == calib_typ['typ1']['calib']: xyz[2] += calib_typ['typ1']['mov_zpoint']
            elif calib['extrinsic'] == calib_typ['typ2']['calib']: xyz[2] += calib_typ['typ2']['mov_zpoint']
            elif calib['extrinsic'] == calib_typ['typ3']['calib']: xyz[2] += calib_typ['typ3']['mov_zpoint']
            elif calib['extrinsic'] == calib_typ['typ4']['calib']: xyz[2] += calib_typ['typ4']['mov_zpoint']

            lwh = list(label[i]['psr']['scale'].values())
            rotation_y = [label[i]['psr']['rotation']['z']]
            class_ = label[i]['obj_type']
            # rename_class = [class_ls[class_]]
            # label_format = ' '.join(map(str, xyz + lwh + rotation_y + rename_class))
            label_format = ' '.join(map(str, xyz + lwh + rotation_y + [class_]))
            labeling_ls.append(label_format)

            # if xyz[0] < -130:
            #     print('x축', scene, frame)

            # if xyz[1] < -90:
            #     print('y축', scene, frame)

            # if xyz[2] > 8:
            #     print('z축', scene, frame)

            x.append(xyz[0])
            y.append(xyz[1])
            z.append(xyz[2])
        except:
            continue

    with open(save_path+f'/{scene}_{frame}.txt', 'w') as f:
        f.write('\n'.join(labeling_ls))

#  객체 포인트 범위 확인
print(f'x축 범위: {min(x)} ~ {max(x)}')
print(f'y축 범위: {min(y)} ~ {max(y)}')
print(f'z축 범위: {min(z)} ~ {max(z)}')

x축 범위: -134.3012173784256 ~ 81.59058380126956
y축 범위: -98.37752972578596 ~ 69.27335041106909
z축 범위: -3.3815337419509883 ~ 10.622396979540696


In [ ]:
len(os.listdir('/NIA50/50-2/data/nia50_all/pvrcnn/labels'))

In [ ]:
# labels = sorted(glob.glob('/data/NIA50/50-2/data/NIA50_real/*/label/*.json'))
# save_path = '/data/NIA50/50-2/data/mot_nia50/pvrcnn/labels/'

# class_list = []
# labels = sorted(glob.glob('/data/NIA50/50-2/data/NIA50_real/*/label/*.json'))
# # save_path = '/data/NIA50/50-2/data/mot_nia50/pvrcnn/labels/'

# df = pd.DataFrame(columns = ['clip', 'CAR', 'PEDESTRIAN', 'MOTORCYCLE'])
# rename = {'Adult': 'PEDESTRIAN', 
#           'Bus': 'CAR',
#           'Car': 'CAR',
#           'Kid': 'PEDESTRIAN',
#           'Kickboard': 'MOTORCYCLE',
#           'Large_Truck': 'CAR',
#           'Light_Car': 'CAR',
#           'Medium_Truck': 'CAR',
#           'Mini_Bus': 'CAR',
#           'Motorcycle': 'MOTORCYCLE',
#           'Pedestrian': 'PEDESTRIAN',
#           'SUV': 'CAR',
#           'Small_Car': 'CAR',
#           'Small_Truck': 'CAR',
#           'Special_Vehicle': 'CAR',
#           'Two_Wheeler': 'MOTORCYCLE',
#           'Van': 'CAR'}

# error_clips = []
# for idx, label in enumerate(labels):
#     try:
#         clip = re.search('Suwon_[A-Z_0-9]+', label).group()
#         frame = re.search('[0-9]+.json', label).group().replace('.json', '')
        
#         with open(label, 'r') as f:
#             label = json.load(f)
        
#         class_list = []
#         # label_txt = []
#         for i in np.arange(len(label)):
#             try:
#                 class_ = label[i]['obj_type']
#                 rename_class = [rename[class_]]
#                 class_list.append(rename_class[0])
#             except KeyError:
#                 print('라벨링 에러 :', clip, frame, class_)
        
#         cnt = Counter(class_list)
#         df.loc[idx] = [clip, cnt['CAR'], cnt['PEDESTRIAN'], cnt['MOTORCYCLE']]
#         # with open(save_path+f'/{clip}_{frame}.txt', 'w') as f:
#         #     f.write('\n'.join(label_txt))
        
#     except ValueError:
#         print('json 에러 :', clip, frame)
#         error_clips.append(clip)
        
# df_ = df.groupby(['clip']).sum().reset_index()

# error_clip_idx = []
# for error_clip in list(set(error_clips)):
#     idx = df_.loc[df_['clip'] == error_clip].index.values
#     error_clip_idx.append(int(idx))

# df_ = df_.drop(error_clip_idx, axis=0)

# for label in labels:
#     try:
#         clip = re.search('Suwon_[A-Z_0-9]+', label).group()
#         frame = re.search('[0-9]+.json', label).group().replace('.json', '')
        
#         with open(label, 'r') as f:
#             label = json.load(f)
        
#         label_txt = []
#         for i in np.arange(len(label)):
#             try:
#             # print(clip)
#             # print(frame)
#                 xyz = list(label[i]['psr']['position'].values())
#                 lwh = list(label[i]['psr']['scale'].values())
#                 rotation_y = [label[i]['psr']['rotation']['z']]
#                 class_ = label[i]['obj_type']
#                 rename_class = [rename[class_]]
#                 # if class_ == 'Special_Vehicle':
#                 #     print(clip, frame)
#                 # class_list.append(class_)
#                 class_list.append(rename_class[0])
#                 label_format = ' '.join(map(str, xyz + lwh + rotation_y + rename_class))
#                 label_txt.append(label_format)
#             except KeyError:
#                 print('라벨링 에러 :', clip, frame, class_)

#         with open(save_path+f'/{clip}_{frame}.txt', 'w') as f:
#             f.write('\n'.join(label_txt))
        
#     except ValueError:
#         print('json 에러 :', clip, frame)


In [ ]:
from collections import Counter

Counter(class_list)

### # ImageSets
---

In [7]:
src_path = '/data/NIA50/50-2/data/NIA50/nia50_all/raw/'
point_src_path = '/data/NIA50/50-2/data/NIA50/nia50_all/pvrcnn_integ/points'
save_path = '/data/NIA50/50-2/data/NIA50/nia50_all/pvrcnn_integ/ImageSets/'
os.makedirs(save_path, exist_ok=True)

# dat_typs = set([re.search('[A-Z]_[A-Z]', j).group() for j in os.listdir(src_path)])
scenes = sorted(os.listdir(src_path))
dat_typs = set([re.findall('[a-zA-Z]+_[A-Z]_[A-Z]', scene)[0] for scene in scenes])
points = sorted(glob.glob(f'{point_src_path}/*.npy'))

train_ls = []
val_ls = []
test_ls = []
for dat_typ in dat_typs:
    # images_typ = [image for image in images if dat_typ in image]
    scenes_typ = [scene for scene in scenes if dat_typ in scene]
    
    train_val, test = train_test_split(scenes_typ, test_size=0.2, shuffle=False, random_state=44)
    train, val = train_test_split(train_val, test_size=0.2, random_state=44)

    # train_ls += [train]
    # val_ls += val
    # test_ls += test
    for j in train:
        for point in points:
            if j in point:
                point = re.findall('[a-zA-Z0-9_]+', point)[-2]
                train_ls.append(point)

    for j in val:
        for point in points:
            if j in point:
                point = re.findall('[a-zA-Z0-9_]+', point)[-2]
                val_ls.append(point)

    for j in test:
        for point in points:
            if j in point:
                point = re.findall('[a-zA-Z0-9_]+', point)[-2]
                test_ls.append(point)

with open(save_path+'train.txt', 'w') as f:
    f.write('\n'.join(sorted(train_ls)))
    
with open(save_path+'val.txt', 'w') as f:
    f.write('\n'.join(sorted(val_ls)))

with open(save_path+'test.txt', 'w') as f:
    f.write('\n'.join(sorted(test_ls)))

In [ ]:
# labels = sorted(glob.glob('/data/NIA50/50-2/data/NIA50_real/*/label/*.json'))
# # save_path = '/data/NIA50/50-2/data/mot_nia50/pvrcnn/labels/'

# df = pd.DataFrame(columns = ['clip', 'CAR', 'PEDESTRIAN', 'MOTORCYCLE'])
# rename = {'Adult': 'PEDESTRIAN', 
#           'Bus': 'CAR',
#           'Car': 'CAR',
#           'Kid': 'PEDESTRIAN',
#           'Kickboard': 'MOTORCYCLE',
#           'Large_Truck': 'CAR',
#           'Light_Car': 'CAR',
#           'Medium_Truck': 'CAR',
#           'Mini_Bus': 'CAR',
#           'Motorcycle': 'MOTORCYCLE',
#           'Pedestrian': 'PEDESTRIAN',
#           'SUV': 'CAR',
#           'Small_Car': 'CAR',
#           'Small_Truck': 'CAR',
#           'Special_Vehicle': 'CAR',
#           'Two_Wheeler': 'MOTORCYCLE',
#           'Van': 'CAR'}

# error_clips = []
# for idx, label in enumerate(labels):
#     try:
#         clip = re.search('Suwon_[A-Z_0-9]+', label).group()
#         frame = re.search('[0-9]+.json', label).group().replace('.json', '')
        
#         with open(label, 'r') as f:
#             label = json.load(f)
        
#         class_list = []
#         # label_txt = []
#         for i in np.arange(len(label)):
#             try:
#                 class_ = label[i]['obj_type']
#                 rename_class = [rename[class_]]
#                 class_list.append(rename_class[0])
#             except KeyError:
#                 print('라벨링 에러 :', clip, frame, class_)
        
#         cnt = Counter(class_list)
#         df.loc[idx] = [clip, cnt['CAR'], cnt['PEDESTRIAN'], cnt['MOTORCYCLE']]
#         # with open(save_path+f'/{clip}_{frame}.txt', 'w') as f:
#         #     f.write('\n'.join(label_txt))
        
#     except ValueError:
#         print('json 에러 :', clip, frame)
#         error_clips.append(clip)
        
# df_ = df.groupby(['clip']).sum().reset_index()

# error_clip_idx = []
# for error_clip in list(set(error_clips)):
#     idx = df_.loc[df_['clip'] == error_clip].index.values
#     error_clip_idx.append(int(idx))

# df_ = df_.drop(error_clip_idx, axis=0)

In [ ]:
# df_.describe()

In [ ]:
# clip1 = list(df_.loc[(df_['PEDESTRIAN'] >= 1) & (df_['MOTORCYCLE'] >= 1), 'clip'])
# clip2 = list(df_.loc[(df_['PEDESTRIAN'] < 1) | (df_['MOTORCYCLE'] < 1), 'clip'])

# train1, val1 = train_test_split(clip1, test_size=0.2, random_state=0)
# train2, val2 = train_test_split(clip2, test_size=0.2, random_state=0)

# train = sorted(train1 + train2); val = sorted(val1 + val2)
# len(train), len(val)

In [ ]:
# save_path = '/data/NIA50/50-2/data/mot_nia50/pvrcnn/ImageSets/'

# frames = [str(i).zfill(4) for i in np.arange(0, 10)]

# train_list = []
# for clip in train:
#     for frame in frames:
#         train_file = clip + f'_{frame}'
#         train_list.append(train_file)

# val_list = []
# for clip in val:
#     for frame in frames:
#         val_file = clip + f'_{frame}'
#         val_list.append(val_file)
        

# with open(save_path + 'train.txt', 'w') as f:
#     f.write('\n'.join(train_list))           
    
# with open(save_path + 'val.txt', 'w') as f:
#     f.write('\n'.join(val_list))           